# Heron Core Demo

In [ ]:
from pynq import Overlay
from pynq import allocate
from tqdm.notebook import tqdm
import numpy as np

Define binary versions of each benchmark. These are generated by the `dumpTemplates` function in the `heron-clash` module. These examples are for an architecture with `-r6:4:2:1:2` parameters.

In [ ]:
codes = dict(
    adjoxo      = [0b000_0010_0010_1000_0000_0001_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0010_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0000_0100_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0001_1000_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_0000_1100_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0010_1000_0011_0000_0010_1010_1011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_0101_0001_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_0001_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1101_0000_0000_0000_1001_1010_0100_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0010_1100_0000_0000_0011_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0010_0101_0001_0000_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1101_1010_0000_0000_0000_1101_0011_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_0000_0000_0000_0000_1101_1000_1000_0000_0000_0001_1010_0000_0000_0000_0001_1010_0010_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_1110_0010_0100_0101_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_0010_0100_0010_0100_1000_1010_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_0101_0001_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0011_0000_0100_1000_0101_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_0000_0100_1000_0110_0011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0110_0000_1001_0000_1100_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_0100_1100_0000_0000_0011_0010_0000_0000_0111_1100_0011_0110_0000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0101_1000_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0100_0000_0100_1000_0100_1010_0000_0000_0111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0001_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0010_1100_0000_0100_1000_0100_1010_0000_0000_1000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0011_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_0100_0100_1000_0100_1010_0000_0000_1000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0100_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_0110_0100_1000_0100_1010_0000_0000_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0100_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0001_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_1000_0100_1000_0100_1010_0000_0000_1000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0100_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_1010_0100_1000_0100_1010_0000_0000_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0011_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0001_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_1100_0100_1000_0100_1010_0000_0000_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0100_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0010_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_1110_0100_1000_0100_1010_0000_0000_1001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0011_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0100_0010_0010_0010_0000_1010_0000_0000_1001_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0101_1000_0000_0000_0001_0110_1001_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1100_1000_0000_0010_0111_1000_1011_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0000_0100_0000_1011_0000_0000_0001_0010_1001_0100_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_0100_0000_0000_0000_0000_1010_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0111_0011_0010_1100_0000_0000_0000_1010_1100_0000_0000_0100_1000_0000_0000_0000_0000_0001_0001_0000_0110_0000_1001_0000_1100_1010_0000_0000_0000_0000_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0100_0110_0101_0000_0010_1010_0001_0000_0100_0101_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_1001_1000_0000_0000_0000_1010_0000_0000_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0010_0001_0100_0001_1101_0000_0100_0001_1001_1001_0101_0000_0000_0000_0011_1010_0010_0000_0000_0001_0010_0000_0110_1000_0000_0000_0100_1101_0000_0000_0000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_1011_1100_0000_0000_0000_1010_0000_0000_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0101_0001_0100_0000_1101_0101_0000_0000_0000_0011_0101_0000_0000_0000_0101_0010_1000_0010_0000_0001_0100_1101_0000_1000_0011_1011_1001_0110_0000_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_0010_0100_0010_0100_1000_1010_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_0101_0001_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_1101_1000_0000_0000_0000_1010_0000_0000_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1101_1000_0000_0000_0001_0011_0000_0000_0000_0100_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1010_1100_0000_0000_0010_0010_0000_1000_0011_0011_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1101_1000_0000_0000_0001_0011_0000_0000_0000_0100_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1011_0100_0000_0000_0010_0010_0000_1000_0011_0011_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_1101_1000_0000_0000_0000_1010_0000_0000_1010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_1101_1000_0000_0000_0000_1010_0000_0000_1010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1011_0000_0000_1001_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1011_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1011_0000_0000_1001_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1011_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1011_0000_0000_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1100_1000_0000_0000_0000_1010_0000_0000_1010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_1101_0000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_1000_1101_0001_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_0001_1101_0100_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0000_0100_1100_1101_0001_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0100_0010_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1000_0100_0001_0100_1100_1010_0111_1111_1111_1111_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0010_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0001_0011_0011_1000_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0010_0100_0101_0000_0010_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0100_1010_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_1000_0011_0000_0100_1000_0111_0011_0001_0000_0000_0000_0010_1100_0000_0000_0000_1010_1001_1111_1111_1111_1111_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_1000_0011_0000_0100_1000_0110_1011_0001_0000_0000_0000_0010_1100_0000_0000_0001_1010_1001_1111_1111_1111_1111_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0100_1010_0000_0000_1000_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_1000_0011_0000_0100_1000_0111_0011_0001_0000_0000_0000_0010_1100_0000_0000_0000_1010_1001_1111_1111_1111_1111_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0100_1010_0000_0000_1000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_1000_0011_0000_0100_1000_0111_0011_0001_0000_0000_0000_0010_1100_0000_0000_0001_1010_1001_1111_1111_1111_1111_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0100_1010_0000_0000_1000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_1000_0011_0000_0100_1000_0111_0011_0001_0000_0000_0000_0010_1100_0000_0000_0001_0010_1001_1111_1111_1111_1111_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0100_1010_0000_0000_1001_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_1000_0011_0000_0100_1000_0111_0011_0001_0000_0000_0000_0010_1100_0000_0000_0000_1010_1001_1111_1111_1111_1111_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0100_1010_0000_0000_1001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_1000_0011_0000_0100_1000_0111_0011_0001_0000_0000_0000_0010_1100_0000_0000_0011_1010_1001_1111_1111_1111_1111_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0100_1010_0000_0000_1001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_1000_0011_0000_0100_1000_0111_0011_0001_0000_0000_0000_0010_1100_0000_0000_0010_0010_1001_1111_1111_1111_1111_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0001_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1000_0100_0001_1101_1100_1011_0000_0000_0000_0000_1101_0000_0000_0000_0000_1000_0100_0001_1101_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0000_0010_0100_1000_0100_1010_0000_0000_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0100_1011_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0011_0000_0100_1000_0110_0010_1001_1111_1111_1111_1011_0110_0100_0000_0000_0010_1001_1111_1111_1111_1110_0001_0000_0000_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1010_0000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1111_1010_1011_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1010_0000_0000_0000_0001_1101_1001_0000_0000_0000_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0111_0111_0010_1100_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0001_0100_1101_0100_1111_1111_1111_1101_0101_1111_1111_1111_1111_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_1000_0100_1000_0101_0000_0110_1010_0000_1000_0011_0011_0011_0110_0100_0000_0000_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_1000_0100_1000_0101_0000_0110_1010_0000_1000_0011_0011_0011_0110_0100_0000_0000_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_1000_0100_1000_0101_0000_0110_1010_0000_1000_0011_0011_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0010_0100_0010_0100_1000_0001_1011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    braun       = [0b000_0000_0100_1000_0000_1001_0000_0100_1010_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0010_1001_0010_1100_0000_0000_0000_0010_1100_0000_0111_1111_1000_0000_0000_0000_0000_0001_0100_1101_0000_1000_0001_0011_1001_0110_0000_0000_0000_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_1000_1100_0110_0000_0000_0000_1101_0011_0000_0000_0000_1010_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_0001_0000_0000_0000_1001_1010_0100_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_1000_1100_0110_0000_0000_0000_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_0000_0100_0000_0000_0000_1010_0000_0000_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1000_0000_0000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0010_0001_0010_0001_1001_1010_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0001_0000_1001_0000_1100_1101_0010_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0011_1000_1000_1000_1001_1010_0110_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1010_0000_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0001_0100_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0001_0010_0101_0000_0110_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_0000_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_1000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0010_1000_0011_0000_0010_1110_1011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_0101_0001_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0011_1000_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1000_0001_0000_0011_0010_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_1010_0101_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_0011_1000_0000_0000_0001_0011_0110_0000_0000_0000_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0001_1001_0010_0001_1001_1011_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0001_0000_1001_0000_1100_1010_0000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0010_0100_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1000_0001_0000_0100_0110_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0000_0100_0100_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_1000_1101_0001_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_0000_1010_1100_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0000_0101_0100_1101_0001_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1010_0000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1001_0001_0010_0001_1001_0100_1111_1111_1111_1111_0000_0100_0001_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0000_0110_0100_1000_0101_1011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0011_1000_1000_1000_1001_0100_1111_1111_1111_1101_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0001_0001_0011_1000_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0001_0101_0011_1000_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    cichelli    = [0b000_0000_1000_1001_0110_1001_0000_0000_1010_0000_0000_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1001_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1000_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0000_1000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0000_0010_0010_0100_0100_1010_0000_0000_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0000_1000_0010_0010_0000_1010_0000_0000_1111_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0010_1000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0001_1000_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0000_1010_0010_0010_0001_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0011_1001_0010_0001_1001_1010_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_0010_0100_1000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0001_0000_0101_0000_0110_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1101_1010_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0101_0000_1001_0001_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0001_1010_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0110_1001_0010_0001_1001_1010_0010_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_0010_1100_0000_0000_0010_1011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0010_1000_0011_0000_0011_1110_1011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_0101_0001_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_0011_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_0100_1000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0010_0000_0010_0110_1011_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0100_0100_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0010_1110_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0100_1100_0010_0000_0000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_0101_0000_0000_0000_0011_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0101_1000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_0010_0100_1000_0101_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0010_1000_0001_0000_0110_0000_1010_0000_0000_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0011_0110_0010_0100_0100_1010_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1000_0000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_0000_0000_0000_0000_1001_1010_0100_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0100_1000_0000_0100_0000_1000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1010_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_0111_0100_0000_0000_0011_0010_0000_0001_0000_0100_0011_0100_1100_0000_0000_0011_0110_1000_0000_0000_0011_0101_0000_0000_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_0111_1100_0000_0000_0010_0011_0110_0000_0000_0000_0011_0100_1000_0000_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1001_0100_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_0101_0000_0000_0000_1101_1010_0110_0000_0000_0001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1000_1100_0000_0000_0011_0010_0000_0001_0000_0110_0011_0110_0000_0000_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0110_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1101_0001_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_1010_0000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0110_1100_0100_0000_0000_0000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0000_0100_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0011_1001_0010_0001_1001_1011_0000_0000_0000_0001_0000_0100_0010_0111_1000_0000_0000_0000_0000_0001_0001_0000_0010_1000_0100_1000_1110_1010_0000_0000_0000_0000_1000_0100_0000_1110_0100_1100_0000_0000_0000_1000,0b111_1001_0000_1011_0000_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b100_1000_0010_0100_0010_0100_1000_0000_1010_0000_0001_0000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0010_1010_0000_0000_0000_1011_0101_0000_0000_0000_0010_1010_0000_0000_0000_1011_0101_0100_0000_0000_0001_0101_0001_0101_0000_0000_0000_0101_1010_0010_0000_0000_0001_0101_0000_0000_0000_0101_1010_0100_0000_0000_0000,0b110_1001_0000_1010_1100_0000_0000_0010_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1101_0000_0000_0000_0000_1101_0010_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0110_1100_0100_0000_0000_0000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_1110_0000_0000_0000_1001_1011_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0011_0100_1001_0000_1100_1101_1001_0000_0000_0000_1010_0000_0000_0000_0101_0000_0000_0000_0000_0000,0b000_0000_0010_0100_0010_0100_1000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0011_0010_0100_1000_0100_1010_0000_0001_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0110_1100_0100_0000_0000_0000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0000_0100_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_0100_0011_0100_1001_1011_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0000_0001_0000_0000_0000_0100_1101_1000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0110_1110_0100_1000_0100_1010_0000_0001_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0011_1001_0000_0000_0000_1101_1011_0010_0000_0000_0001_1010_0000_0000_0000_0001_0100_0000_0000_0000_0011_0100_1101_0000_1000_0100_1110_1001_0110_0000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0100_1100_0010_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_1101_1000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1110_0100_0000_0000_0010_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0000_1110_0100_1000_0111_0010_0000_0001_0001_1110_0010_1000_0000_0000_0000_0011_0101_0000_0000_0000_0010_1000_0000_0000_0000_1010_1000_0000_0000_0000_1110_1000_0000_0000_0001_0010_0000_0101_0100_1001_0000_0000_1001_1011_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0000_0010_1010_1000_1000_0000_0100_1101_1000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_1111_0000_0000_0000_0000_1010_0000_0001_0010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1101_0001_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0000_1100_0100_1000_0101_0010_1000_0000_0000_0000_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_1001_1111_0010_1100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_0000_1100_0100_0001_1001_1010_0010_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_0000_0000_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_1000_0111_1110_0100_1000_0100_1010_0000_0001_0010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0100_1000_0000_0000_0000_1001_1010_0110_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_0000_1000_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0001_0010_0000_0010_0100_0110_1011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_0101_0001_0000_0000_0011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0001_0010_0000_0010_0100_0110_1011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_0101_0001_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1000_0000_0100_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0011_0000_0011_0001_0001_0111_0010_0000_0001_0010_1010_0010_0101_0001_0000_0000_0011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_1000_1100_0110_1000_0111_0010_0000_0001_0010_1100_0010_1000_0000_0000_0000_0010_1000_0000_0000_0000_0111_0101_0000_0000_0000_0011_0110_1100_0000_0000_0011_0110_1000_0000_0000_0010_0010_0010_0010_0000_1000_1000_1001_1010_0010_0000_0000_0001_1011_0110_0000_0000_0000_0000_0000_0000_0000_0001_0001_0001_0000_1100_0100_0100_0100_1101_0000_0000_0000_0000_1101_1011_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_1100_1000_0000_0100_1011_1000_1001_0010_1000_0000_0000_1000_0100_0010_1001_0100_1101_1100_0000_0000_0000_1001_0100_0000_0100_0000_1001_0100_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0001_0010_1000_0000_0000_0001_1010_0000_1000_0101_0011_0010_1000_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0011_0001_0010_0001_1001_1010_0000_0000_0000_0001_0000_0100_0011_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1000_0100_0001_1010_1100_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0111_1001_0010_0001_1001_1010_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_1001_1010_1001_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_1001_1010_1001_0000_0011_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_0101_0000_0001_0000_0011_0100_1000_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0001_0011_0000_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0000_1000_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_1000_1101_0001_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_0010_1000_0100_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0000_0111_0100_1101_0001_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_1000_1101_0000_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_0010_1001_0100_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0100_1000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0010_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1100_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0000_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1000_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1001_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1100_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0000_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1011_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0001_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1000_0101_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1000_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1101_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0110_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1101_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0000_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0010_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0010_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1000_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0111_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1000_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1100_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0111_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1010_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1010_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0110_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0110_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1101_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1011_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0110_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1110_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1101_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0010_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1011_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0011_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1011_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0010_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0100_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0100_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1011_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0000_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1101_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1000_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1011_1001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0010_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1010_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1110_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1001_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0100_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1101_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1110_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1001_0110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1100_0111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_0110_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1010_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_0011_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1000_1111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1110_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_0111_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1000_0010_0010_0100_1100_1010_0000_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1110_1100_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_0101_1111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_1100_0000_0100_1000_0110_0011_0110_0100_0000_0000_0010_1100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0001_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0001_1101_0011_1000_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0001_0010_0100_1000_0010_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0001_0010_0100_1000_0010_0011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1100_0000_0000_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_1010_0000_0000_0000_1101_1000_0000_0000_0000_1001_0100_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_1010_0000_0000_0000_1101_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0100_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_1000_0101_0101_0010_1011_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1100_0010_0010_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0101_0000_1001_0001_1101_1011_0000_0000_0000_0001_0100_1111_1111_1111_1111_1000_0000_0000_0001_0001_0100_1101_0000_1000_0100_1110_1001_0110_0000_0000_0000_0001_0101_1111_1111_1111_1010_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0100_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0101_1011_0010_0001_0000_1010_0011_0011_0101_1000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_0100_1111_1111_1111_1101_0010_1000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1010_0111_1111_1111_1111_1010_0000_0000_0000_0000_1001_0100_0100_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1111_0110_0101_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0100_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0110_1001_0010_0001_1001_1011_0010_0000_0000_0001_0101_0000_0000_0000_0100_0000_0000_0000_0000_0001_0001_0000_0010_1000_0100_1000_1110_1010_0111_1111_1111_1110_1000_0100_0000_1101_0100_1101_0010_0000_0000_0000,0b000_0010_0010_1000_0000_0100_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0110_0011_0010_1011_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1000_0011_0000_1001_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0111_1001_0010_0001_1001_0100_1111_1111_1111_0111_0101_1111_1111_1111_1010_0000_0000_0000_0000_0001_0001_0000_0001_0100_0100_0100_0100_1010_0111_1111_1111_1100_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_1100_1000_0000_0000_0001_1011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0010_1010_1100_1000_0000_0000_0000_1101_1000_0000_0000_0000_1010_0111_1111_1111_1110_1010_0111_1111_1111_1111_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_1001_1101_0010_1100_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0100_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0011_0001_1000_0000_0000_0011_0000_0000_0000_0010_0011_0000_0000_0000_0010_0011_0000_0000_0000_0010_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1010_0000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1111_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0000_1110_0100_1000_0100_1010_0000_0001_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0111_0111_0011_0110_1000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_0000_1100_0011_1110_1001_1011_0110_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0000_1110_0100_1000_0101_0010_0001_1000_0111_0111_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_0111_0001_0011_0110_1000_0000_0000_0011_0110_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_1110_0100_1000_0100_1010_0000_0001_0010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_0001_0011_0010_0100_0000_0011_0011_0110_1000_0000_0000_0010_1100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0011_0110_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1001_0001_0001_1100_0000_0000_0010_0011_0100_0000_0000_0000_0010_1001_1111_1111_1111_1111_0100_1000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1100_0010_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0001_0111_0001_1000_1100_0000_0000_0001_0100_1111_1111_1111_1101_0100_1111_1111_1111_1111_0100_0000_0000_0000_0000,0b010_1010_1100_1101_1000_0000_0000_0000_1011_0000_0000_0000_0010_1001_0100_0000_0100_0000_1101_1001_0000_0000_0000_1001_0100_0000_1000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0001_0010_0100_1000_0001_0011_0110_1000_0000_0000_0011_0110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1010_1100_1101_0011_0000_0000_0000_1101_0001_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_0010_1001_0100_1101_1100_0000_0000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    clausify    = [0b000_0000_1000_1000_0100_1001_0000_0000_1010_0000_0000_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0110_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1101_0000_1000_0100_0101_1001_0101_0000_0000_0000_0001_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0000_0100_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0010_1100_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_1111_0000_0000_0000_1001_1010_0100_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0001_0000_0010_1100_0000_0000_0000_1010_0000_0000_1001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0000_1100_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_1111_0000_0000_0000_1001_1010_0010_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0001_1100_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_1100_0101_0000_0001_1011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_1100_0101_0000_0001_1011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0001_1010_0101_1000_0111_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0011_0110_1100_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0011_0110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1111_0000_0000_0000_1001_1010_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0000_0111_1000_0000_0000_0100_1101_0001_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_0100_1100_0000_0000_0000_1010_0000_0000_1001_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1111_0000_0000_0000_1001_1010_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1100_0010_0000_0001_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1100_0010_0000_0001_1000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_0011_0000_0000_0000_1001_1010_0000_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0001_0010_0000_1001_1000_0000_0000_0100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_0101_1100_0000_0000_0001_0011_0100_1000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0101_1100_0000_0000_0001_0011_0100_0100_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0100_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0101_1100_0000_0000_0001_0011_0100_0100_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0110_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_0011_0000_0000_0000_1001_1011_0100_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0000_1001_1000_0000_0000_0100_1101_1010_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_1000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_1000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0100_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_1000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0110_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_1111_0000_0000_0000_1001_1010_0100_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0110_1100_0000_0000_0011_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_0101_0001_0000_0000_0011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0001_0000_0000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0111_0100_0000_0000_0011_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0100_0110_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0001_1001_0010_0001_1001_1010_0010_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1000_0100_0000_0000_0010_1011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_1000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_1000_0000_0000_0000_1101_1000_0100_0000_0010_0001_1010_0000_0000_0000_0001_0110_0000_0000_0000_0001_0010_0001_0100_0000_0000_0000_0110_1100_0010_0000_0001_0000_1101_0001_0000_0000_0000_1011_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_1000_0000_0000_0000_1101_1000_0100_0000_0010_0001_1010_0000_0000_0000_0001_0110_0000_0000_0000_0001_0010_0001_0100_0000_0000_0000_0110_1100_0010_0000_0001_0000_1101_0001_0000_0000_0000_1011_0000_0000_0000_0000,0b111_1001_0000_1010_0000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1100_0010_0000_0001_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0110_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_1000_0000_0000_0000_1001_1010_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0001_0100_0000_0000_0000_0100_1101_0001_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_1000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_1000_0000_0000_0000_1001_1010_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0001_0100_0000_0000_0000_0100_1101_0001_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1001_0000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1100_0010_0000_0001_1000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1011_1000_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0001_0000_1011_0011_0011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0101_1010_0101_1000_0010_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0010_1100_0000_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1101_0000_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_1011_1000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_0001_0000_0110_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_0010_1000_0001_0000_1100_1000_1010_0000_0000_1001_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1100_1100_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_1101_1000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0010_1100_0000_0100_1000_0100_1010_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0001_1001_0010_0001_0000_0110_0001_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_1000_1000_0010_0010_0100_0111_0010_0000_0000_1010_0010_0011_0000_0000_0000_0010_0010_1000_0000_0000_0000_0110_0000_1000_0110_1011_0010_1000_0000_0000_0000_1111_0000_0000_0000_0010_0010_1101_1011_0001_0000_0000_0000_0011_0000_0000_0000_0010_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1001_0000_1000_0000_0010_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_1110_0000_0000_0000_0001_0011_0100_0100_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0011_1000_0000_0000_0000_1001_1010_0000_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0100_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0110_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1110_0000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0100_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_0100_0010_0100_1000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0111_1000_0010_0010_0000_1010_0000_0000_1010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0001_1001_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_0000_1100_0011_1101_1001_0000_0100_0011_1110_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1101_1010_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0010_0000_1000_1001_0001_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_1000_0100_1001_0000_0000_1010_0000_0000_1010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_1100_1000_0000_0010_1100_0000_1101_0000_0000_0000_0000_1000_0100_0010_0100_0100_1011_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_1000_1000_0010_0010_0000_1010_0000_0000_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_0000_0000_0000_0110_1000_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_0010_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0100_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0001_0001_0100_0000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_1000_1110_0101_0000_0110_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_0010_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_1000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_1000_1000_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0010_0011_0100_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_1000_1011_0010_1011_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0100_1101_0000_1000_0100_0101_1001_0101_1111_1111_1111_1101_0101_1111_1111_1111_1100_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_1000_1011_0010_1011_1111_1111_1111_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0100_1101_0000_1000_0100_0101_1001_0100_1111_1111_1111_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1000_0010_0001_1011_1100_1010_0000_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_1001_0011_0010_1100_0000_0000_0001_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0001_0001_0000_0100_0100_1000_1110_1010_0000_0000_0000_0000_1100_0100_0000_0000_0000_1010_1111_1111_1111_1010,0b010_0001_0000_0010_1100_0000_0000_0010_1011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0011_1100_0000_0000_0001_1011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0000_1011_1000_0000_0000_0010_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0010_0001_0000_0110_0001_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0100_0110_0100_1000_0101_1011_0100_0100_0000_0000_0010_1001_1111_1111_1111_1110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0100_0110_0100_1000_0100_1010_0000_0000_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_1000_0111_1111_0010_0001_0000_0110_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_0000_1100_0011_1101_1001_0100_0000_0000_0000_0001_0000_0100_0100_0000_1000_0000_0000_0000_0000_0000,0b000_0000_1000_0000_0010_0100_1000_0100_1010_0000_0000_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_1110_0000_0000_0000_0000_1010_0000_0000_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_0011_1100_0000_0000_0000_1010_0000_0000_1010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_1010_0000_0000_0000_0001_1011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_1001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0001_1001_0010_0001_1001_1010_0100_0000_0000_0001_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0001_0000_1100_0000_0000_0010_1011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1000_0100_0010_0100_0100_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0001_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0111_1111_1111_1111_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0100_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0001_0001_0100_0000_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_1000_1101_0011_1000_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    countdown   = [0b000_0010_0010_1000_0000_0001_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1001_0000_0000_0000_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_1101_0100_0000_0000_0000_1001_0100_1001_0100_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0001_0011_0010_0100_0000_0011_0010_0000_0000_0111_1110_0010_0001_0000_0110_1111_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_0010_0100_0010_0100_1000_0010_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_0101_0001_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0001_0011_0000_0100_0000_0010_1011_0100_0000_0000_0000_0010_0100_1010_0000_0000_0010_0001_0000_0110_1111_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_1000_1101_0000_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_0001_1100_1100_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0110_1000_0010_0010_0000_1010_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1001_0000_0000_0000_1001_1010_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_1000_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0011_0000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_0010_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1100_0000_0000_0000_1001_1011_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1011_0000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_0011_1100_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0110_1100_0100_1000_0100_1010_0000_0000_1000_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0001_0111_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_0010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_0010_0000_0000_0000_1001_1011_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1011_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0010_0010_0101_0000_0100_1010_0000_0000_1000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0110_1100_0100_1000_0100_1010_0000_0000_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0001_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0000_0000_1000_0100_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_0101_0000_0000_0000_0000_1010_0000_0000_1001_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_0110_0000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0011_0010_0101_0000_0100_1010_0000_0000_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1011_0001_0010_0001_1001_1010_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_1000_0110_1000_0010_0010_0000_1010_0000_0000_1001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0110_0100_1010_0000_1110_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_1101_1011_0000_0000_0000,0b111_1001_0000_0111_0000_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_1000_0110_1100_0100_1000_0100_1010_0000_0000_1001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_1000_0011_1011_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1000_0000_0000_0001_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0110_1100_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0011_0100_0011_0000_0111_1011_0010_0000_0000_1001_1100_0011_0111_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1010_0010_1000_0000_0010_0111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0011_0000_0000_1000_0100_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0001_0010_0000_0010_1000_0000_0000_0110_1101_1000_0000_0000_0000_1101_0011_0000_0000_0000_1101_0100_0000_0000_0000,0b111_0000_1000_0100_0110_0100_1000_0100_1010_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0100_0110_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0001_1001_0010_0001_1001_1010_0010_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1000_0000_0000_0000_0011_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0010_0101_0000_0100_0000_0011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1000_1000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_1010_1000_0000_0000_0000_1010_0000_0000_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_1000_0000_0000_0110_1100_1000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1011_0000_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_0010_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_0000_0000_0000_0110_1000_1000_0000_0000_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_0010_1001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0101_1000_0000_0000_0001_0110_1000_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0001_1011_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0010_0101_0000_0010_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1001_1100_0000_0000_0001_1010_0000_0000_1010_1000_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0110_1100_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1001_0100_0000_0000_0010_1011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_0101_0001_0000_0000_0011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_1010_0100_0000_0000_0010_0010_0001_0000_0100_1001_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0101_1100_0010_0100_0110_1011_0110_0100_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0100_0000_0011_0110_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0001_0001_0000_1001_0000_0011_0010_0000_0000_1010_1010_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_1000_0000_0000_0010_0101_0000_0001_0000_0011_0011_0110_1101_0000_0000_0000_0110_1101_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_1011_0100_0000_0000_0010_1011_0100_0000_0000_0000_0010_0100_1010_0000_0000_0010_0001_0000_0110_1111_0010_1100_0000_0000_0001_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1100_0100_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_1010_0110_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1100_0000_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0110_0110_0010_0010_0000_1010_0000_0000_1010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1010_0000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_1101_0100_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0110_1000_0010_0010_0001_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1011_0001_0010_0001_1001_1010_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_1000_1101_0001_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_0001_1100_0100_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0001_0010_0100_1101_0001_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_1000_1101_0001_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_0001_1101_0100_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0001_0110_0100_1101_0001_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0110_0110_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0001_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1011_0001_0010_0001_1001_0100_1111_1111_1111_1111_0000_0100_0000_1110_1000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0010_1001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0001_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1001_0000_1000_0001_1111_1001_0110_0000_0001_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1000_0100_0001_1101_1100_1011_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1011_0001_0010_0001_1001_0100_1111_1111_1111_1001_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0000_0000_0000_0000_1001_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1010_1111_1111_1111_1110_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1010_1001_0010_0001_1001_0101_1111_1111_1111_1101_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1001_1000_1000_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0111_1111_1111_1110_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1010_1001_0010_0001_1001_0100_1111_1111_1111_1001_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0011_1100_0000_0000_0001_1011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0110_0100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_1000_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1110_0000_1000_0110_0101_0000_0000_0000_0000_0000_0001_0110_1001_0000_1000_0010_1111_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0111_1111_1111_1100_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1010_1001_0010_0001_1001_0100_1111_1111_1111_0101_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_1100_0000_0100_1000_0110_0011_0110_0100_0000_0000_0010_1100_0000_0000_0000_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0000_0100_1000_0000_0000_0010_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0010_0000_1000_0010_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_1100_0000_0100_1000_0110_0011_0110_0100_0000_0000_0010_1100_0000_0000_0000_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0010_1100_0100_1000_0110_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0010_0001_0000_0011_0101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0110_1100_0100_1000_0101_1011_0110_1100_0000_0000_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0000_0000_0000_0100_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1000_0000_0000_0011_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1100_0100_0000_0000_0000_1100_0000_0000_0000_0000_1010_0111_1111_1111_1111_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1111_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0000_0100_0000_0000_0001_1011_0111_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0010_1100_0100_1000_0101_1011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1010_1100_1110_1000_0000_0000_0000_1101_1000_0000_0000_0000_1001_0100_0100_0000_0000_1101_1001_0000_0000_0000_1101_1000_0000_0000_0000_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1011_0000_0000_0000_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1110_0000_0000_0000_0000_1110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1101_1010_0000_0000_0000_1101_1001_0000_0000_0000_1001_0100_0100_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1000_0100_0001_0110_0100_1110_0000_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0001_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    fib         = [0b000_1010_0100_1000_0010_0000_0001_1100_1011_0000_0000_0010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0000_0100_1001_0000_0110_0011_0110_0000_0000_0000_0010_1100_0000_0000_0001_0010_0101_0001_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1000_0000_0000_0010_0000_1001_0100_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0101_1000_0000_0000_0010_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0001_1011_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0010_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1000_0010_0000_0000_1100_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1000_0010_0000_0000_1100_1110_0000_0000_0000_0000_1000_0010_0000_0000_1100_1110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    knuthbendix = [0b000_0001_0001_0000_0100_0000_0000_0000_1010_0000_0010_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_0101_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_1001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b111_1000_0010_0100_0000_0010_0010_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_0100_0010_0100_1000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0000_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0010_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0100_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0010_0000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1101_0010_0000_0000_0000_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1101_0000_0000_0000_0000_1101_0010_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0010_1100_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_1010_0110_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0011_1000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1000_0001_0000_0011_0110_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0010_0010_0010_0010_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1000_1000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_1000_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0010_0010_0010_0010_0001_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1001_1001_0010_0001_1001_1010_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0010_1100_0101_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0010_1100_0101_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0010_1010_0101_1000_0111_0011_0110_1000_0000_0000_0011_0110_1100_0000_0000_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0110_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_0110_0000_0000_0000_0000_1010_0000_0010_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_1000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0010_1110_0010_0110_1011_0011_0110_1000_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0011_1000_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1110_0001_0010_0001_1001_1010_0010_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_0110_1000_0000_0000_0010_1011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1101_1010_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1110_1000_1001_0001_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0000_1111_1000_0000_0000_0110_1101_0100_0000_0000_0000_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000,0b111_1000_1000_0011_1100_0101_1000_0110_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0010_1100_0010_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0010_0010_0010_0010_0001_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0000_1001_0010_0001_1001_1010_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0010_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0100_0100_0010_0010_0010_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0100_0110_1001_0000_0010_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_0000_1000_0100_1001_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1001_1000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0100_1010_0010_0100_0100_1010_0000_0010_0101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0010_1000_0011_0000_1010_0110_1011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_0101_0001_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_1001_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_1010_0010_0100_0101_1011_0100_1000_0000_0000_0010_1000_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_1000_0101_1001_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0101_1010_0010_0100_0100_1010_0000_0010_0101_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1000_0000_1001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1000_0001_0010_0001_1101_1000_1000_0000_0000_0001_1010_0000_0000_0000_0001_1010_0010_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_1011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1011_1000_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_1100_1100_1010_0000_1110_1101_0011_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0000,0b111_0000_1000_0110_0110_0101_0000_0101_1011_0100_1100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1100_0100_0000_0000_0011_0010_0000_0010_0101_1010_0011_0110_0000_0000_0000_0011_0110_0000_0000_0000_0011_0110_1100_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0110_1110_0101_0000_0101_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0100_1100_0010_0000_0000_0000_1101_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_1101_0000_0000_0000_0011_0010_0000_0010_0101_1100_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0110_1100_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1101_1000_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1110_1100_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1110_1100_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_1000_0001_0000_1110_0101_1010_0000_0010_0101_1110_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_1000_0000_0000_0110_1000_0000_0000_0000_0100_1010_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0111_0100_0101_0000_0001_1011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0010_0010_0010_0010_0000_1010_0000_0010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1001_0001_0010_0001_1001_1010_0000_0000_0000_0001_0000_0100_0100_0111_1000_0000_0000_0000_0000_0001_0001_0000_0100_1000_1001_0000_1100_1010_0000_0000_0000_0000_1010_0000_0000_0000_1010_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0000_1001_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0101_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_1000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_0110_0000_0000_0000_0100_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0100_0100_0010_0010_0000_1010_0000_0010_0111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0101_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1011_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0100_0100_0010_0010_0000_1010_0000_0010_1000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0101_0111_0000_0000_0000_1101_1011_0010_0000_0000_0001_0110_0000_0000_0000_0001_0100_0000_0000_0011_0001_0100_1101_0100_0000_0000_0000_0001_1011_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b111_1000_1000_1000_1100_0101_0000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0001_0011_0000_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0011_1010_0010_0100_0100_1010_0000_0010_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_1000_0111_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0010_1000_0001_0001_0001_0101_1010_0000_0010_1010_0110_0011_0110_1000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0101_1000_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0001_0010_0100_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_1010_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0101_0000_1010_0000_0000_0000_1001_1011_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_1100_1001_0000_1100_1010_1000_0000_0000_0010_1000_0010_1000_0111_1100_0000_0000_0000_0000_0000,0b110_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0001_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1001_0001_0010_1000_0000_0000_0000_1010_0000_0010_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_0010_1100_0000_0000_0010_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_0011_0100_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1000_0000_1010_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_1111_0100_0000_0000_0000_1001_1010_0100_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0111_1010_0000_0000_0000_0100_1101_0011_0000_0000_0000_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0001_0100_0100_0000_0000_0000_1010_0000_0010_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0001_0101_0010_0000_1000_0000_1011_0011_0000_1000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1001_0000_1100_0000_0110_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_0101_1100_0000_0000_0010_1011_0100_1100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_1000_1000_0000_1010_1110_0000_1101_0100_0000_0000_0000_1101_0000_0000_0000_0000_1101_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_1000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_1010_1000_0110_1000_0111_0010_0000_0010_1011_1010_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_0001_0001_0101_0100_0000_0000_0000_1010_0000_0010_1011_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_1010_1100_0111_0000_0111_0010_0000_0010_1011_1110_0011_0100_0000_0000_0000_0011_0101_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0100_1100_0010_0000_0000_0000_1101_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_0110_1000_0000_0000_0011_0010_0000_0010_1100_0000_0010_1000_0000_0000_0000_0111_0110_0000_0000_0000_0010_1010_0000_0000_0000_0011_0100_1100_0000_0000_0010_1000_0000_0000_0000_1010_1101_1011_0001_0000_0000_0000_0011_0111_0000_0000_0000_0011_0111_0100_0000_0000_0000_0000_0000_0000_0000_0001_0111_0001_0001_0100_0101_0011_1001_1011_0000_0000_0000_0001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000,0b100_1010_0010_1000_0000_1011_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1000_1000_1000_1000_1001_1010_0000_0000_0000_0001_1010_1100_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_0100_0100_0100_0100_1101_0101_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1100_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_1000_0011_0001_0110_1111_0010_0001_1001_0000_1011_0010_0000_0000_0111_1111_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1100_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0001_0111_0000_0000_0000_0011_0010_0001_1001_0000_1011_0010_0000_0000_0111_1111_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0010_0010_0010_0010_0000_1010_0000_0010_1100_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_1110_0101_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_1100_1001_0000_1100_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1001_0001_0010_0001_1101_1000_1000_0000_0000_0001_1011_0100_0000_0000_0001_1011_0010_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_0100_0000_0000_0000_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_1000_1100_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_0100_0110_0010_1101_0000_1000_1110_0111_1001_1010_0100_0000_0000_0001_1010_0010_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0011_1000_0100_1000_0100_1010_0000_0010_1100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_1100_1011_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1000_0000_0111_1001_0000_1000_1111_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_1010_0000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_0100_0110_0110_1101_0000_1000_1110_0111_1001_1011_0100_0000_0000_0001_1010_0000_0000_0000_0001_0000_0010_0110_0010_0011_0011_1110_1000_0100_0111_0011_1100_1101_1010_0000_0000_0000_1101_0001_0000_0000_0000,0b110_1000_0010_0100_0010_0100_1000_0000_1010_0000_0010_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0101_0000_1100_0000_0000_0000_1001_0101_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0000_0010_0110_0000_1000_0000_0100_1101_0010_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1010_0010_1000_0000_1011_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0101_0000_1100_0000_0000_0000_1001_1011_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_1100_1001_0000_1100_1010_0000_0000_0000_0000_1010_0000_0000_0000_0011_0000_0000_0000_0000_0000,0b110_1010_0110_1100_0100_0000_0000_0000_1101_0011_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0001_1010_1100_0000_0000_0011_0010_0000_0010_1101_1100_0011_0110_1000_0000_0000_0011_0110_1100_0000_0000_0011_0111_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_0000_1100_1110_1010_1001_0100_0000_0000_0000_0001_1011_0110_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1001_0001_1011_0100_0000_0000_0000_1010_0000_0010_1110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_1011_1000_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0001_1110_0000_0000_0000_0011_0011_0001_0000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_1000_0000_0000_0000_0111_0100_1000_0000_0000_0011_0000_0000_0000_0000_0010_0100_1000_0010_0000_0000_0000_1101_1000_1000_0000_0000_0001_1011_0000_0000_0000_0001_1011_0010_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_1101_0000_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_0010_0010_0010_0011_0010_0000_0010_1110_0110_0011_0111_0000_0000_0000_0010_1000_0000_0000_0000_0111_0110_1000_0000_0000_0011_0000_0000_0000_0010_0010_1000_0000_0000_0000_1110_0100_0111_1000_0000_0000_0000_1101_1000_1000_0000_0000_0001_1011_0000_0000_0000_0001_1011_0010_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1011_0000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_1100_1100_0000_0000_0010_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_1000_0100_1000_0100_1010_0000_0010_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_0101_0010_0000_1000_0001_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0100_0011_0000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_0000_1000_0010_0110_0100_1000_0100_1010_0000_0010_1110_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1010_0000_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_1101_1100_0000_0000_0010_1011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_0000_0000_0000_0000_0010_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_0000_1000_1111_1000_0101_0000_0100_1010_0000_0010_1110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_0100_0111_1010_1101_0000_1000_1110_0111_1001_1011_0010_0000_0000_0001_0100_0000_0000_0000_0101_0000_0010_0110_0010_0011_1101_1110_1000_0100_0111_0011_1100_1101_1001_0000_0000_0000_1010_0000_0000_0000_0101,0b111_0000_1000_1111_1010_0110_1000_0111_0010_0000_0010_1111_0010_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0101_0100_0000_0000_0011_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_1111_1100_0111_1000_0111_0010_0000_0010_1111_0100_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0011_0111_0000_0000_0000_0011_0111_0100_0000_0000_0010_0010_1000_0010_0000_1000_1000_1001_1011_1000_0000_0000_0001_1011_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_1111_1100_0000_0000_0010_1011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0011_0101_0000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_1011_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1110_0001_0010_0001_1001_0100_0000_0000_0000_0111_0100_0000_0000_0000_1100_0000_0000_0000_0000_0001_0001_0000_0100_1100_1001_0000_1100_1010_1000_0000_0000_0010_1000_0010_1000_0111_1100_0000_0000_0000_0000_0000,0b000_0010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1001_0001_0110_0111_1000_0111_0010_0000_0011_0000_0000_0010_1010_0000_0000_0001_1010_1010_0000_0000_0001_1111_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0010_0010_0000_1001_0001_0010_0001_1101_0000_1000_1010_0001_1001_1011_0010_0000_0000_0001_0101_0000_0000_0000_1011_0001_0000_0100_1000_1001_0000_1110_1000_0100_0101_0000_1100_1101_1010_0000_0000_0000_1010_1000_0000_0000_0101,0b111_0000_1000_0011_1010_0010_0100_0100_1010_0000_0011_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0011_1010_0010_0100_0100_1010_0000_0011_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0010_0100_0100_1000_0101_0011_0100_0100_0000_0000_0010_0000_1000_0000_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_0111_1100_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1001_0001_0010_0101_1000_0100_1010_0000_0011_0000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_1000_1000_0000_0000_0000_1001_1011_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0001_0011_0000_0000_0000_0100_1010_0000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_0010_1100_0001_0010_0010_1010_0010_0000_0011_0000_1110_0011_0101_0000_0000_0000_0011_0101_0100_0000_0000_0011_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_0000_0000_0000_0110_1001_1000_0000_0000_0100_1010_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_0011_1100_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0001_0010_0001_0010_0011_0100_1010_0000_0011_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0011_1010_0010_0100_0111_0010_0000_0011_0001_0010_0011_0100_0100_0000_0000_0010_1000_0000_0000_0000_0010_0001_0000_0101_0101_0010_1100_0000_0000_0000_0010_0101_0000_0001_0000_0010_1101_0010_0001_0001_0001_1001_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0001_0011_0000_0100_0000_0001_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1110_1100_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_0011_0100_0101_0000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0010_1000_0001_0010_0100_1000_1010_0000_0011_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0001_0010_0100_1111_0010_0000_0011_0001_1000_0011_0100_1100_0000_0000_0011_0111_0000_0000_0000_0011_0100_0100_0000_0000_0011_0111_0000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_0010_1000_0101_1000_0010_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_0011_0100_0101_0000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0010_1000_0001_0010_0101_1000_1010_0000_0011_0001_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_0010_1100_0001_0010_0101_1100_1010_0000_0011_0001_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_0110_0000_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0010_0100_0100_1000_0101_0011_0100_0100_0000_0000_0010_0000_1000_0000_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1001_0010_1100_0000_0000_0000_0000_1010_0000_0011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0010_1011_0000_0000_0000_0001_1011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_1001_0000_1011_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0010_0111_0000_0000_0000_0011_0010_0000_0011_0010_0010_0011_0100_1100_0000_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_0011_1100_0110_0000_0010_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_1100_0000_0101_0000_0001_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0011_1010_0010_0100_0100_1010_0000_0011_0010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0001_0100_0101_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_0000_0100_0010_0100_1001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_1100_0011_0010_1000_1100_1010_0000_0011_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_0101_0000_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0001_0001_0010_1010_0000_1001_0000_1100_1101_0011_0000_0000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0100_1000_0000_1100_1010_1000_1101_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_1000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0001_0010_1001_0100_0000_0000_0000_1010_0000_0011_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_1001_1100_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0010_1011_0000_0000_0000_0001_1011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0010_1010_0100_0000_0000_0011_0010_0000_0011_0010_1110_0011_0100_1100_0000_0000_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_0101_0110_0101_1000_0010_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0010_1100_0000_0100_1000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_1100_0000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_0010_1100_0001_0010_1011_1000_1010_0000_0011_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_1001_0101_1110_0110_0000_0100_1010_0000_0011_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_0110_1110_0010_0100_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0001_0011_0010_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0010_0010_0010_0010_0000_1010_0000_0011_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1000_1000_1000_1000_1101_1000_1000_0000_0000_0001_0110_0000_0000_1010_0001_1000_0000_0000_0001_0001_0001_0000_0100_1000_1001_0000_1100_1101_0010_0000_0000_0000_1010_0000_0000_0000_0100_0000_0000_0000_0000_0000,0b110_1000_1001_0110_1000_0010_0110_0000_1010_0000_0011_0011_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0010_0010_0010_0010_0000_1010_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1000_1000_1000_1000_1101_1000_1000_0000_0000_0001_0110_0000_0000_1000_0001_1000_0000_0000_0001_0001_0001_0000_0100_0100_0100_0100_0110_1100_0100_0000_0000_0000_1011_0000_0000_0100_0000_1100_0000_0000_0000_1000,0b001_0001_0010_1101_0100_0000_0000_0011_0010_0000_0011_0100_1010_0011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_1110_0100_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0010_0010_0010_0010_0000_1010_0000_0011_0100_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1000_1000_1000_1000_1101_1000_1000_0000_0000_0001_0110_0000_0000_1000_0001_0100_0000_0000_0000_0101_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_1000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b111_1001_0010_1110_1100_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0010_0010_0010_0010_0000_1010_0000_0011_0101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1000_1000_1000_1000_1101_1000_1000_0000_0000_0001_0110_0000_0000_1000_0001_0100_0000_0000_0000_0111_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_1000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b111_1001_0011_0000_0100_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_1000_0000_0100_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_1000_0000_0100_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1000_0001_0010_1111_1010_1010_0001_0010_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1001_0111_1110_0110_0000_0111_0010_0000_0011_0101_0110_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_0000_0000_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0011_0101_0100_0000_0000_0010_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0000_0000_0000_0010_0010_1000_0000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1001_0011_0101_0100_0000_0000_0000_1010_0000_0011_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1000_1000_1000_1000_1001_1010_0100_0000_0000_0001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0011_0100_0011_0011_0000_1111_0010_0000_0011_0101_1010_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0011_0101_0100_0000_0000_0011_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_0001_0011_0101_0100_0000_0000_0000_1010_0000_0011_0101_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0010_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_0001_0000_0000_0000_0011_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0011_0101_0100_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1000_0100_1100_1000_1001_1010_0100_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1101_0000_1000_1101_0111_1001_0100_0000_0000_0000_0001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1001_0011_0101_0100_0000_0000_0000_1010_0000_0011_0101_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b101_0000_0011_0000_0011_0011_0010_0100_1010_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_1001_0100_0110_1000_0111_0010_0000_0011_0110_0010_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0111_0000_0000_0000_0011_0111_0100_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0011_0010_1100_0000_0000_0011_0010_0000_0011_0110_0110_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0101_0100_0000_0000_0011_0110_0100_0000_0000_0011_0101_1000_0000_0000_0010_1101_0011_0000_1000_0000_0010_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1001_0011_0101_0100_0000_0000_0001_1011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0011_0011_0100_0000_0000_0011_0010_0000_0011_0110_1000_0010_1000_0000_0000_0000_1111_0100_1100_0000_0000_0011_0100_0100_0000_0000_0011_0111_0000_0000_0000_0010_1010_0000_0000_0000_0010_0100_1111_0100_0000_0000_0000_1001_1011_0000_0000_0000_0001_1011_1000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0111_1010_0000_0000_0000_0110_1100_0010_0000_0000_1000_1101_1010_0000_0000_0000_1101_1100_0000_0000_0000,0b110_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_0001_1000_0000_0000_0010_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_0011_1100_0000_0000_0010_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1100_0010_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_0100_0100_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_0100_1100_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0010_0010_0010_0010_0000_1010_0000_0011_0110_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0001_1101_1111_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_0000_1100_0000_1010_1001_0000_0100_0000_0101_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1001_0011_1001_0100_0000_0000_0000_1010_0000_0011_0110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1000_1000_1000_1000_1001_1010_0100_0000_0000_0001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0011_0100_0011_0011_0110_0011_0010_0000_0011_0110_1110_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0011_0101_0100_0000_0000_0011_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_1011_0010_0111_0000_0111_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_1001_0100_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0011_1001_0100_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1000_0010_1100_0011_0011_0111_0000_1010_0000_0011_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_1111_0100_0000_0000_0000_1101_1000_0100_0000_0001_0001_1010_0100_0000_0000_0001_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0011_0111_0100_0000_0000_0011_0010_0000_0011_0111_0010_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0011_0110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_0110_1000_0000_0000_0010_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_0111_1100_0000_0000_0010_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1100_0010_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_1000_0100_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_1000_1100_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0011_1010_0000_0000_0000_0000_1010_0000_0011_0111_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1001_1101_0101_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0010_0110_0100_1000_0100_1010_0000_0011_0111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1001_1101_0101_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1000_1110_0111_1001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1101_0001_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1001_1101_0110_0101_1000_0100_1010_0000_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0010_0110_0100_1000_0101_0011_0100_1000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1001_1101_1001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_1011_1000_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0001_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_1111_0100_0000_0000_0000_1001_1010_0010_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0100_0010_0000_0100_1000_1110_1101_0010_0000_0000_0000_1101_0011_0000_0000_0000_1010_0000_0000_0000_0000,0b000_1000_1001_1101_1010_0101_1000_0111_0010_0000_0011_1000_0100_0010_1000_0000_0000_0000_0111_0000_0000_0000_0010_0011_0100_0100_0000_0000_0011_0110_1100_0000_0000_0011_0110_1000_0000_0000_0010_0010_1000_0010_0000_1000_1000_1001_1011_0110_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b111_1001_0011_1101_0000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_1101_0000_0000_0000_0001_1011_0000_1000_0000_0010_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_1001_1110_0000_0010_0110_1000_1010_0000_0011_1000_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_1100_0100_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1100_0010_0000_0000_1000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0001_1101_1111_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_1000_1001_0000_1100_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_1100_1000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_1110_1000_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0011_1010_0010_0100_0100_1010_0000_0011_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_0010_1000_0001_0011_1101_1100_1010_0000_0011_1000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1110_1100_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_1111_0000_0101_1000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0010_1100_0001_0011_1110_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1000_0000_1110_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1101_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0011_1111_0000_0000_0000_0011_0010_0000_0011_1000_1110_0010_0101_0000_0100_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_1000_0001_0100_0000_0111_0010_0000_0011_1001_0000_0010_1000_0000_0000_0000_0011_0000_0000_0000_0010_0010_1100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_0100_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_1000_0010_0100_1000_0000_1010_0000_0011_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1010_0000_1000_0010_0010_0001_0011_0100_1100_0000_0000_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0011_0100_1000_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0010_0000_0010_0100_1010_0000_0000_0001_0100_1101_0100_0000_0000_0000_0001_0000_1001_0001_0000_1001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0100_0000_1000_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1010_0001_0000_0010_0100_0101_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1010_0010_1000_0000_1110_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0011_0100_1000_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0010_0000_0010_0100_1010_0000_0000_0001_0110_1101_0000_1101_0000_0101_1001_1010_0110_0000_0000_0001_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0100_0001_1000_0000_0000_0010_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0100_0010_1000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0011_1010_0010_0100_0100_1010_0000_0011_1001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_1000_0101_0111_0010_0001_0000_0111_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_1110_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0001_1010_0100_0000_1001_1011_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1101_0000_1001_0001_0001_1001_0110_0000_0000_0000_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0010_0100_0100_1000_0100_1010_0000_0011_1001_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0011_0110_0100_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0010_0000_0010_0100_1010_0000_0000_0001_0100_1101_0100_0000_0000_0000_0001_0000_1001_0001_0000_1001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0010_0100_0100_1000_0100_1010_0000_0011_1001_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_1101_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0110_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0110_0000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_1000_1101_0001_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_1000_1001_0100_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0001_0100_0100_1101_0001_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0011_1111_1000_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0011_1010_0010_0100_0100_1010_0000_0010_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_1011_0101_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_0100_0100_1000_0100_1010_0000_0010_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_1000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1110_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_1010_1101_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_1000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_1100_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0001_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1110_0110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0110_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_0101_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_1011_0101_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_1100_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0001_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_1100_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0001_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1101_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_1100_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1011_1101_0100_1111_1111_1100_1100_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1011_1110_1001_1111_1111_1111_1000_0000_0000_0000_0000_0001_0110_1001_0000_1000_0100_0011_1001_0101_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1010_1100_1010_1111_1111_1111_1110_1010_0000_0000_0000_0000_1000_0100_0000_0000_1100_1100_0000_0000_0000_1000_1010_1111_1111_1111_1101_1010_1111_1111_1111_1110_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0001_1001_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0000_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_1100_0000_0100_1000_0110_0011_0110_0100_0000_0000_0010_1100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0101_0001_0011_1000_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_1000_0010_1100_0101_0000_0010_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1100_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0010_1100_0101_0000_0001_0011_0110_1100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_1010_0000_0000_0000_1101_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0101_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1100_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1001_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0001_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1000_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1010_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0100_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1100_0100_0000_0000_0000_1011_0000_0000_0001_0100_1010_0111_1111_1111_1111_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0001_0111_0101_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_1000_0100_1001_0000_1100_1010_0111_1111_1111_0111_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1001_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1101_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0110_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1010_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1011_0000_0000_0001_0100_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0000_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0001_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_0101_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_0110_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1011_0000_0000_0000_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1110_0110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1010_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1000_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1010_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1100_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0010_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_1000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0110_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1100_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_0110_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1011_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1100_0100_0000_0000_0000_1011_0000_0000_1001_0010_1010_0000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0110_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0011_1101_0000_0000_0000_1001_0100_1111_1111_1111_1111_1011_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_1000_1001_0000_1100_1101_1001_0000_0000_0000_1010_0000_0000_0001_0011_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0000_1001_0010_0001_1001_0100_1111_1111_1111_1111_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0010_1001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_1101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1100_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0111_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1011_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0100_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1101_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0000_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1101_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1000_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0111_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1010_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_0111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_1001_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1001_0000_1000_1011_1000_1001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1001_1001_0010_0001_1001_0101_0000_0000_0000_0011_0000_0101_0000_1111_1000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1000_0000_0000_0001_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_1100_0100_0000_1001_0101_1111_1111_1111_1111_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_1000_1000_0100_0100_0100_1010_0111_1111_1110_1010_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1100_0100_0000_0000_0000_1011_0000_0000_1010_1010_1010_0111_1111_1111_1010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0001_0000_1000_1000_1001_0100_1111_1111_1101_0101_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0010_0100_0100_1000_0110_0011_0100_0100_0000_0000_0010_1001_1111_1111_1111_1110_0001_0000_0000_0011_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0001_1010_0100_0000_1101_1011_0010_0000_0000_0001_0110_0000_0000_0000_0001_1011_0100_0000_0000_0001_0001_0000_1101_1100_1010_0000_1110_1010_0000_0000_0000_0000_1000_0100_0001_1100_0100_1101_0000_0000_0000_0000,0b000_0010_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0010_0010_0000_1000_1000_1101_0100_1111_1111_1111_1111_1100_0000_0000_0000_0001_0010_1000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0000_0000_0000_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0000_0010_0110_0110_0010_0100_0100_1010_1000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1010_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1001_0001_0010_0001_1001_0100_1111_1111_1111_1111_1000_0100_0000_0001_0000_0000_0000_0000_0000_0001_0010_0000_1111_1000_0000_0000_0110_1010_0111_1111_1111_1100_1010_0000_0000_0000_0000_1100_0100_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_1111_0100_0000_0000_0000_1001_1011_0000_0000_0000_0001_0101_1111_1111_1111_1110_0000_0000_0000_0000_0001_0010_0111_1010_0000_0000_0000_0100_1101_0001_0000_0000_0000_1010_1111_1111_1111_1111_0000_0000_0000_0000_0000,0b010_1001_0001_0011_1100_0000_0000_0011_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0011_0110_1000_0000_0000_0011_0101_0000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0001_0011_1100_0000_0000_0000_1010_0000_0010_1011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1001_0100_0010_1000_0000_0000_0011_0011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0100_1000_0000_0000_0010_1001_1111_1111_1111_1111_0110_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1000_1100_0010_1100_1100_1101_0001_0000_0000_0000_1101_0010_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0001_0111_1000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1001_0001_0100_1000_0000_0000_0011_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1101_1010_0000_0000_0000_1011_0000_0111_1101_0000_1001_0100_0001_0000_0000_1101_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0001_1000_0000_0000_0000_0000_1010_0000_0010_1100_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_1011_1111_0010_1011_1111_1111_1111_1011_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1001_0001_1011_1100_0000_0000_0010_0011_0111_0100_0000_0000_0011_0110_0000_0000_0000_0011_0111_0000_0000_0000_0011_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_0010_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0010_0001_1000_1100_1001_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0010_1001_1111_1111_1111_1101_0001_0000_1100_1100_1010_0000_1110_1101_0001_0000_0000_0000_1000_0100_0101_1110_0100_1100_0000_0000_0000_1000,0b001_0001_0001_1100_0100_0000_0000_0001_1011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1011_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1000_0110_0001_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1001_1001_0100_0001_1101_0100_1111_1111_1111_1011_0000_1000_1011_1100_1001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0110_0110_0101_0000_0100_1010_0000_0010_1100_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_0101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_1000_0010_0100_0100_1000_0110_1011_0100_1000_0000_0000_0010_1001_1111_1111_1111_0110_0001_0001_0111_1001_0011_0000_0000_0000_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1111_1010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_0100_1111_1111_1111_1111_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0100_1010_0000_0010_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1010_0000_0000_0000_0001_1011_0000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1001_1101_0101_0010_1001_1111_1111_1111_1110_1011_1111_1111_1111_0100_0000_0000_0000_0000_0001_0001_0000_0100_1100_1001_0000_1100_1010_0111_1111_1111_1100_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_0000_1100_1110_1010_1001_0100_0000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_0100_0110_1010_1001_0100_1111_1111_1111_1011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_1000_1101_0010_0101_1000_0111_0010_0000_0010_1101_1110_0010_1001_1111_1111_1111_1110_1100_0000_0000_0000_0011_0110_1100_0000_0000_0011_0111_0000_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0100_1010_0000_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_0110_0100_1000_0100_1010_0000_0010_1110_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0100_0011_0000_0000_0000_0001_0011_0110_1100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1001_0001_1011_1100_0000_0000_0011_0011_0101_0000_0000_0000_0011_0100_1000_0000_0000_0011_0110_1100_0000_0000_0011_0110_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0001_1110_0000_0000_0000_0001_0011_0001_0000_0000_0000_0011_0110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1100_0000_0000_0011_0111_0000_0000_0000_0000_0000_0000_0000_0000_0001_0101_0001_0100_1111_1111_1111_1101_0100_0000_0000_0000_0001_1011_0100_0000_0000_0001_1000_0000_0000_0000_0000,0b001_0001_0100_0011_0000_0000_0000_0001_1011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0001_1101_0100_0000_0000_0001_1011_0101_0100_0000_0000_0011_0110_1100_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_1111_0011_0011_0100_1000_0000_0000_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0011_1110_1000_0000_0000_0000_1010_0000_0010_1110_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_1111_0100_0000_0000_0000_1001_1010_0100_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0111_1010_0000_0000_0000_0100_1101_0101_0000_0000_0000_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1011_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_1111_0100_0000_0000_0000_1001_1010_0110_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0100_0010_0000_0100_1000_1110_1101_0100_0000_0000_0000_1010_0111_1111_1111_1111_1010_0000_0000_0000_0000,0b001_1010_1000_1010_1000_0000_0000_0000_1010_1000_0000_0000_0001_1010_1000_0000_0000_0000_1010_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1110_1010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_0100_1111_1111_1111_0111_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1000_0100_0110_0001_0100_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_1100_0000_0101_0000_0110_0010_1011_1111_1111_1111_1110_1100_0000_0000_0000_0010_1011_1111_1111_1111_1111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1011_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0000_0000_0000_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0000_0010_0110_0110_0100_0000_1100_1010_1000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1011_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_1000_0010_1001_0010_0001_0000_0111_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_0000_1100_0000_1001_1001_0100_0000_0000_0000_0001_0101_0000_0000_0000_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1011_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_0101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_1000_1001_0000_1100_1010_0111_1111_1111_1010_1100_0010_0000_0000_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1011_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0101_0000_1010_0000_0000_0000_1001_1011_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0000_1111_1000_0000_0000_0110_1010_1000_0000_0000_0000_1010_0111_1111_1111_1111_1100_0100_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_1111_0100_0000_0000_0000_1001_1011_0000_0000_0000_0001_0101_1111_1111_1111_1110_0000_0000_0000_0000_0001_0010_0111_1010_0000_0000_0000_0100_1101_0001_0000_0000_0000_1010_1111_1111_1111_1111_0000_0000_0000_0000_0000,0b010_1000_0010_1000_0001_0010_0001_1011_0010_0000_0011_0000_0010_0010_1100_0000_0000_0000_0010_1010_0000_0000_0001_1110_1010_0000_0000_0001_1010_1010_0000_0000_0001_1010_1010_0000_0000_0001_1110_0101_0000_1010_0000_0000_0000_1001_1011_0100_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1100_0010_1100_1001_0000_1000_0011_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_0100_0010_0100_1000_0000_1010_0000_0011_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0101_0000_1010_0000_0000_0000_1001_1011_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0111_0100_0100_1000_1110_1010_0111_1111_1111_1110_1010_0111_1111_1111_1111_1010_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1100_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_1000_1111_0000_0000_0000_1001_1011_0010_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0100_0111_1000_0000_0000_0100_1101_1010_0000_0000_0000_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1010_1111_1111_1111_1110_1010_1111_1111_1111_1111_1001_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1110_0010_0000_1000_0000_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_0100_1111_1111_1111_0011_0000_0100_0000_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0000_0111_1100_0000_0000_0010_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_0001_0001_0000_0111_0010_0001_0000_0000_0011_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0000_0111_1100_0000_0000_0010_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_0001_0001_0000_1001_0010_0001_0000_0000_0011_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_1001_0010_1010_0101_1000_0011_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0010_1001_1111_1111_1111_1111_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_1010_0000_0100_1000_0111_1101_1100_0000_0000_0000_0001_1010_0000_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0110_1110_0101_0000_0110_0011_0100_0100_0000_0000_0010_0001_0000_0111_0001_0011_0100_1000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_0100_0100_1000_0100_1010_0000_0011_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0010_0011_1000_0000_0000_0001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_1100_0000_0100_1000_0110_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1000_0001_0010_0100_0100_1010_0000_0011_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_0010_1100_0000_0101_0000_0101_0011_0100_1000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_1100_0000_0100_1000_0110_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0000_1110_1100_0000_0000_0010_1011_0100_1000_0000_0000_0011_0110_1100_0000_0000_0011_0100_0100_0000_0000_0011_0110_1100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1001_0010_0110_1100_0000_0000_0010_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1110_1100_0000_0000_0001_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_0110_0100_1000_0100_1010_0000_0011_0010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_1111_0000_0000_0000_1101_1010_0010_0000_0000_0001_0100_1111_1111_1111_1111_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0100_0011_0000_0000_0000_0010_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_0110_0001_0001_0100_1001_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1010_0000_1000_0010_0010_0001_1011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1001_0000_1110_1100_0000_0000_0011_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0011_1110_1000_0000_0000_0001_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_0100_0010_0100_1000_0000_1010_0000_0011_0011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1001_0000_1110_1100_0000_0000_0011_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0010_1100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1001_0000_1110_1100_0000_0000_0011_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1100_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1000_0001_0010_0001_1001_0100_1111_1111_1111_1111_0100_0000_0000_0000_0110_0000_0000_0000_0000_0001_0001_0000_0111_0100_0100_1000_1110_1010_0000_0000_0000_0000_1000_0100_0000_1000_0100_1100_0000_0000_0000_1000,0b000_0010_0010_1000_0000_1100_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0001_0110_0101_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_1011_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1100_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_0100_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0001_0000_0100_0100_0100_0100_0100_1010_0111_1111_1111_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1110_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_0010_1100_0000_0100_1000_0110_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1101_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0010_0010_0001_0010_1100_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0010_1101_1100_0100_1000_0100_1010_0000_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0010_0010_0001_0010_1100_0000_0000_0000_1011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0010_1101_1100_0100_1000_0100_1010_0000_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1101_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_0100_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0001_0000_0100_0100_0100_0100_0100_1010_0111_1111_1111_1111_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1110_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_0100_0100_0100_0100_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1100_0100_0000_0000_0000_1011_0000_0000_0101_0000_1100_0000_0000_0000_1000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1101_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_0100_0100_0100_0100_1010_0111_1111_1111_1001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1001_0110_0110_0100_1000_0000_1010_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0110_1110_0101_0000_0101_0011_0110_1000_0000_0000_0010_0001_0000_0111_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1001_0110_1110_0010_0100_0000_1010_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_1110_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0001_0010_1101_1100_0100_1000_0100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_0000_0000_0000_0000_1101_1010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1111_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1001_0110_1110_0010_0100_0000_1010_0000_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0001_1111_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_1011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_0000_0000_0000_0000_1101_1010_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0101_1011_1000_1001_0000_1001_1010_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_0100_1111_1111_1111_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_1000_0001_0010_1111_0110_0010_0001_0010_0000_0001_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0010_0010_0010_0010_0010_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1001_1111_1111_1111_1111_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1110_1100_0000_0000_0001_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0010_1010_1100_0100_0000_0000_0000_1010_0111_1111_1111_1110_1101_0100_0000_0000_0000_1010_0000_0000_0000_0000_1101_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0001_1010_1111_0010_1001_1111_1111_1111_1111_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_0010_1000_0010_0100_1000_0011_0011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0110_0000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_1100_0101_0000_0000_1010_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1001_0100_0010_1000_0000_0000_0010_0011_0111_0100_0000_0000_0010_1100_0000_0000_0000_0010_0001_0000_0111_0001_0011_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0011_1110_1000_0000_0000_0001_1011_0100_0000_0000_0000_0010_1001_1111_1111_1111_1111_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1001_0011_1110_1000_0000_0000_0010_0011_0110_0000_0000_0000_0010_1000_0000_0000_0000_0010_1011_1111_1111_1111_1111_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0010_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1101_0000_1000_1100_0010_1001_0101_1111_1111_1111_1111_0101_1111_1111_1111_1100_0000_0000_0000_0000_0000,0b000_1010_0100_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1001_0001_0010_0001_1001_1010_0010_0000_0000_0001_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0010_0010_0010_0010_0010_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1001_1111_1111_1111_1111_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1110_1100_0000_0000_0001_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0011_1110_1000_0000_0000_0010_1011_0100_1100_0000_0000_0010_1001_1111_1111_1111_1111_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_1100_0101_0000_0000_1010_0000_0011_0111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0010_0100_0100_1000_0110_0011_0110_1100_0000_0000_0010_0000_1000_0000_0111_0010_0001_0000_0111_0001_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0100_1010_0000_0011_0111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_0110_0100_1000_0100_1010_0000_0011_0111_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1001_0100_0011_0000_0000_0000_0011_0011_0110_0100_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1110_1100_0000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_0110_0100_1000_0100_1010_0000_0011_0111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1001_0100_0011_0000_0000_0000_0010_0011_0110_0100_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0101_0000_0100_1010_0000_0011_1000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_1010_0000_1000_0010_0010_0011_0011_0110_1000_0000_0000_0011_0110_0100_0000_0000_0010_1100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0011_1001_0100_0000_0000_0001_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0000_1110_1100_0000_0000_0010_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0000_0111_1100_0000_0000_0010_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_0001_0001_1110_1101_0010_0001_0000_0000_0011_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1001_0000_1110_1100_0000_0000_0010_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_1111_1110_0000_0000_0000_1101_1010_0110_0000_0000_0001_1100_0000_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_1010_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1000_0010_0100_1000_0000_1010_0000_0011_1001_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0011_1001_0100_0000_0000_0001_1011_0001_0000_0000_0000_0011_0110_0000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1001_0011_1001_0100_0000_0000_0011_0011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0000_0000_0000_0010_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0010_1001_1111_1111_1111_1010_0001_1010_0010_0111_0010_1001_1111_1111_1111_1111_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0010_0100_0100_1000_0110_0011_0100_0100_0000_0000_0010_0000_1010_0001_0011_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0010_0001_1101_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_1100_1001_0000_1100_1010_0111_1111_1111_1111_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0101_0000_1100_0000_0000_0000_1001_0100_1111_1111_1111_1111_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_1000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_1100_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0101_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1110_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_1011_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0101_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1110_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_1010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0100_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1110_1001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_1001_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0100_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1110_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_1000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0011_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1110_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_0111_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1110_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_0110_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0010_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1110_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_0101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0010_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_0100_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0001_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1110_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_0011_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1110_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0000_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1100_0100_0000_0000_0000_1011_0000_0000_1000_0010_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_0001_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_0000_1100_0000_1001_1001_1000_1000_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    mate        = [0b000_0001_0010_1101_1100_0000_0000_0000_1010_0000_0001_0111_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0100_0010_1100_0000_0000_0011_1000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1000_0000_0000_0001_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_0010_1001_0100_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_0010_1001_0100_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0010_0000_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_1010_0110_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0001_1100_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0100_1000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0010_1000_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_0010_0010_0010_0001_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0101_1001_0010_0001_1001_1010_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_1010_0100_1000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0001_1000_0101_0000_0110_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1101_1010_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0111_0000_1001_0001_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0001_1110_0010_0010_0000_1010_0000_0001_0111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_0000_0000_0000_0110_1000_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_0001_0000_0000_0000_1001_1010_0100_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0000_0010_0110_0000_1000_0000_0100_1010_1000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_0100_0000_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0010_1100_0010_0110_1010_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0100_1100_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0100_1100_0000_0000_0011_0010_0000_0001_1000_0100_0011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0101_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0101_0100_0000_0000_0010_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_0010_0010_0100_0101_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0010_1110_0101_1000_0010_1011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0110_0000_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_0000_0000_0000_0000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_0000_0000_0000_0000_1001_1010_0000_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0110_1000_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0110_1100_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_0000_0000_0000_0000_1001_1010_0100_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1000_1000_0011_1100_0010_0110_1000_1010_0000_0001_1000_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_0111_1100_0000_0000_0010_0011_0110_0000_0000_0000_0011_0100_1000_0000_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0001_1101_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1000_1000_0000_0000_0010_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1001_0000_0000_0000_0010_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1011_0000_0000_1000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1011_0000_0000_1001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1011_0000_0000_1001_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1011_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1011_0000_0000_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1011_0000_0000_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1011_1000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0101_1000_0010_0100_0100_1010_0000_0001_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_1011_0100_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_0010_0100_0000_0010_0010_0000_1010_0000_0001_1000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_0000_0000_0000_0110_1000_0000_0000_0000_0100_1010_0000_1000_0000_0000_0000_0000_0000_0000_0001_1011_1011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0010_0101_0000_0100_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1100_0000_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0010_0100_0000_0010_0010_0000_1010_0000_0001_1000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0101_1000_0000_0000_0001_0110_1100_0000_0000_0000_0100_1010_0010_0000_0000_0000_0000_0000_0000_0000_0001_1011_1011_0110_0000_0000_0000_0010_1100_0000_0000_0100_0010_0101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_0100_0001_0000_1100_1001_0010_0000_0001_1000_1110_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0101_1000_0000_0000_0001_0110_1100_0000_0000_0000_0100_1010_0010_0000_0000_0000_0000_0000_0000_0000_0001_1011_1011_0110_0000_0000_0000_0010_1100_0000_0000_0100_0010_0101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0001_0110_0100_1000_0100_1010_0000_0001_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_0111_1001_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0110_0010_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0110_0010_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0110_0010_0111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0110_0010_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0110_0011_0111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0110_0010_0001_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_1101_0100_0000_0000_0010_1011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_0000_0100_0000_0000_0010_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_1001_0100_0000_0000_0001_1011_0100_1000_0000_0000_0010_1000_0000_0000_0000_0111_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_1001_0000_1001_0010_0000_1001_1010_0110_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0000_1110_0000_0000_0000_0110_1010_0000_0000_0000_0000_1101_0100_0000_0000_0000_1101_1001_0000_0000_0000,0b111_0001_0000_1001_0100_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_0010_0010_1000_0000_0110_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0011_0001_1000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0001_0101_0001_1000_0000_0100_0010_0001_1010_0000_0000_0000_0001_1010_0010_0000_0000_0001_0100_0000_0000_0000_0000,0b111_0000_1000_0111_1110_0111_0000_0111_0010_0000_0001_1001_0100_0011_0111_0000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0010_1010_0000_0000_0001_0010_0100_0011_1101_0000_0000_0000_1101_1011_0010_0000_0000_0001_1011_0000_0000_0000_0001_0100_0000_0000_0000_0011_0000_0010_0100_0000_0100_0100_0100_1101_1010_0000_0000_0000_1010_1000_0000_0000_0011_0000_0000_0000_0000_0000,0b111_1001_0001_0000_0000_0000_0000_0010_1011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0001_0010_0010_0010_0000_1010_0000_0001_1001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_1000_1001_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0001_0111_0001_0001_0000_0100_0100_1001_1011_0100_0000_0000_0001_1011_0000_0000_0000_0001_1011_0010_0000_0000_0000,0b110_1000_1000_0001_0010_0010_0010_0000_1010_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_1000_1001_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0001_0111_0001_0001_0000_0100_0100_1001_1011_0100_0000_0000_0001_1011_0000_0000_0000_0001_1011_0010_0000_0000_0000,0b110_0000_0011_0000_0011_0001_0010_1100_1010_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_1100_0010_0110_1010_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0000_0000_0000_0010_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1011_0000_1001_1010_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1000_0000_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_0100_0010_0100_1000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_0100_0000_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0001_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0111_0001_1000_1100_0000_0000_0001_1010_0000_0000_0000_0001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000,0b101_1000_1000_1001_0000_0101_0000_0100_1010_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1000_0000_0110_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_1000_1001_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0001_0111_0001_1000_1100_0000_0000_0001_1011_0010_0000_0000_0001_1000_0000_0000_0001_0001_1000_0000_0000_0001_0000,0b110_0001_0001_0010_0100_0000_0000_0000_1010_0000_0001_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_0011_0100_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_1011_0010_0010_0110_1010_0010_0000_0001_1011_1010_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0001_1011_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0010_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_0011_1100_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_1011_0010_0010_0110_1010_0010_0000_0001_1100_1100_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0001_1011_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0010_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_1100_0010_0110_1010_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0000_0000_0000_0010_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1011_0000_1001_1010_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1000_0000_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_0100_0010_0100_1000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_0100_0000_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_0000_1000_1011_0010_0010_0110_1000_1010_0000_0001_1110_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0111_0001_1000_1100_0000_0000_0001_1010_0100_0000_0000_0001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000,0b110_1000_1000_1011_0010_0010_0110_1010_0011_0101_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_0100_1100_0000_0000_0011_0010_0000_0001_1110_0100_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0110_0000_0000_0000_0011_0101_0000_0000_0000_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_1011_0010_0010_0110_1010_0011_0101_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0011_0001_1000_0000_0000_0011_0100_1000_0000_0000_0011_0000_0000_0000_0010_0011_0000_0000_0000_0010_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_1011_0010_0010_0110_1010_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0001_0101_0100_0000_0000_0011_0010_0000_0001_1110_1000_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0111_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0001_0101_1100_0000_0000_0011_0010_0000_0001_1110_1010_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0001_1000_1100_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1011_0000_0000_0000_0000_1011_0000_0000_0000_0010_1001_0100_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_1100_0010_0110_1010_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0000_0000_0000_0010_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1011_0000_1001_1010_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1000_0000_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_1110_1001_0100_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_0100_1001_0100_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_1100_0010_0110_1010_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0000_0000_0000_0010_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1011_0000_1001_1010_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1000_0000_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_1000_0000_0100_1000_0100_1010_0000_0001_1110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0011_0001_1000_0000_0000_0011_0100_0000_0000_0000_0011_0000_0000_0000_0010_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1000_1000_0001_0010_0010_0010_0000_1010_0000_0001_1110_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0101_1101_0000_0000_0000_1001_1011_0000_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0001_0000_0101_0000_0010_0011_0000_0110_1010_0000_0000_0000_0000_1101_0011_0000_0000_0000_1101_1100_0000_0000_0000,0b110_0000_1000_0001_0010_0010_0010_0000_1010_0000_0001_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0001_1011_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0010_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_1100_0010_0110_1010_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0000_0000_0000_0010_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1011_0000_1001_1010_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1000_0000_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_0100_0010_0100_1000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_0100_0000_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_1100_0010_0110_1010_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0000_0000_0000_0010_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1011_0000_1001_1010_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1000_0000_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_1000_1000_0110_0011_0101_0100_1101_0000_0000_0000_0000_1101_1001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0110_0111_0000_0000_0000_1001_1010_0100_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_1000_1101_0000_0101_1000_0100_1010_0000_0010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0001_1011_0100_0000_0000_0000_1010_0000_0010_0000_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_1001_0000_0000_1001_1011_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_0010_1001_0100_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0001_0000_1001_0100_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0011_0001_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0001_0100_0100_1000_0100_1010_0000_0010_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1000_0000_0000_0011_0000_0000_0000_1000_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0001_0010_0010_0010_0010_1010_0001_1000_1000_0101_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0010_0001_1000_1000_0111_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0001_1010_0100_1000_0000_1010_0000_0010_0001_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_1110_0111_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0010_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0000_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_0100_1100_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0010_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0000_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_0010_0010_0100_0101_1011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1100_1000_1001_0001_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0001_0001_1101_1000_0000_0000_0000_1010_0000_0010_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_1000_0000_0000_0110_1001_0000_0000_0000_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_0010_0010_0100_0101_1011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1100_1000_1001_0001_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0001_0001_1110_0000_0000_0000_0000_1010_0000_0010_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_1000_0000_0000_0110_1001_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0011_0000_0001_0001_1110_0110_1010_1000_0000_0000_0000_0111_0100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0011_0110_1100_0000_0000_0000_0000_0000_0000_0000_0010_1010_0011_0110_1100_0000_0000_0011_0101_0000_0000_0000_0010_0101_0000_0001_0000_0011_0110_0100_0000_0000_0001_0101_0001_0100_0000_0000_0000_0001_1011_0100_0000_0000_0001_0010_1000_0000_1000_0001_0010_1000_0010_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_0000_1001_0101_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_0000_1001_0101_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_0000_1001_0101_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_0000_1001_0101_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_0000_1001_0101_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_0000_1001_0101_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_0010_0100_0000_0010_0010_0000_1010_0000_0010_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_0000_0000_0000_0110_1000_1000_0000_0000_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0001_1011_1011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0010_0101_0000_0010_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1101_0000_0000_0000_0000_1011_0000_0000_0000_0000_1001_0101_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0010_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0010_0000_0001_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0010_0100_1100_0000_0000_0010_1011_0001_0000_0000_0000_0011_0000_0000_0000_0000_0010_1000_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_0010_0010_0100_0101_1011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1100_1000_1001_0001_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0010_0001_1000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0001_0010_1001_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_0010_0010_0100_0101_1011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1100_1000_1001_0001_1101_1010_0010_0000_0000_0001_1011_0100_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0010_0010_0000_0000_0000_0001_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0001_0010_1111_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0011_0000_0001_0010_0010_0110_1010_0000_0010_0001_1110_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1010_0000_0000_0000_0110_1100_1000_0000_0000_0100_1010_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_0000_1000_1111_0100_0110_1000_0100_1010_0000_0010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0011_0110_1100_0000_0000_0011_0111_0000_0000_0000_0010_0101_0000_0010_0000_0011_0110_0100_0000_0000_0001_0000_0110_0000_0010_0011_1011_1110_1010_0000_0000_0000_0010_1101_1000_0000_0000_0000_1101_1001_0000_0000_0000,0b101_0000_0010_0100_0000_0010_0010_0000_1010_0000_0010_0010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_1000_0000_0000_0110_1000_1000_0000_0000_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0001_1011_1011_0101_0000_0000_0000_0011_0100_1000_0000_0000_0010_0101_0000_0010_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1001_0000_0010_0110_0000_0110_1010_0000_0010_0010_0110_0011_0110_1100_0000_0000_0011_0110_0100_0000_0000_0011_0111_0000_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1101_1000_0000_0000_0110_1100_1000_0000_0000_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0001_1011_1011_0111_0000_0000_0000_0011_0110_1000_0000_0000_0010_0101_0000_0010_0000_0000_0000_0000_0000_0000_0000,0b101_0000_0010_0100_0000_0010_0010_0000_1010_0000_0010_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_1000_0000_0000_0110_1000_1000_0000_0000_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_0000_1010_0110_0000_0111_0010_0000_0010_0011_0010_0011_0110_0000_0000_0000_0011_0111_0100_0000_0000_0011_0110_0000_0000_0000_0011_0110_1100_0000_0000_0011_0111_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1100_0000_0000_0011_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_0001_0100_0110_1000_0111_0010_0000_0010_0011_0100_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0110_1100_0000_0000_0011_0110_0100_0000_0000_0011_0111_0000_0000_0000_0011_0011_0110_1101_1000_0000_0000_0110_1100_1000_0000_0000_0100_1010_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0010_0010_1100_0000_0000_0011_0010_0000_0010_0011_0110_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0101_0000_0000_0000_0011_0101_0100_0000_0000_0011_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0010_0100_0100_0000_0000_0011_0010_0000_0010_0011_1000_0011_0101_0000_0000_0000_0011_0110_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_0100_1000_0000_0000_0010_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_0101_1000_0000_0000_0010_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0010_0100_0000_0010_0010_0000_1010_0000_0010_0011_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_0000_0000_0000_0110_1100_1000_0000_0000_0100_1010_0010_0000_0000_0000_0000_0000_0000_0000_0001_1011_1011_0110_0100_0000_0000_0011_0100_1000_0000_0000_0010_0101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1000_0010_1100_0001_0010_0101_0100_1010_0000_0010_0011_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1101_1000_0000_0000_0110_1110_0000_0000_0000_0100_1010_0010_0000_0000_0000_0000_0000_0000_0000_0001_1011_1011_0110_1100_0000_0000_0011_0111_0000_0000_0000_0010_0101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_0110_0100_0000_0000_0010_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0010_0100_0000_0010_0010_0000_1010_0000_0010_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_0000_0000_0000_0110_1100_1000_0000_0000_0100_1010_0010_0000_0000_0000_0000_0000_0000_0000_0001_1011_1011_0110_0100_0000_0000_0011_0100_1000_0000_0000_0010_0101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b101_1000_0010_1100_0001_0010_0110_0000_1010_0000_0010_0100_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1101_1000_0000_0000_0110_1110_0000_0000_0000_0100_1010_0010_0000_0000_0000_0000_0000_0000_0000_0001_1011_1011_0110_1100_0000_0000_0011_0111_0000_0000_0000_0010_0101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_0111_0000_0000_0000_0010_1011_0101_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0010_0100_0000_0010_0010_0000_1010_0000_0010_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_0000_0000_0000_0110_1100_1000_0000_0000_0100_1010_0010_0000_0000_0000_0000_0000_0000_0000_0001_1011_1011_0110_0100_0000_0000_0011_0100_1000_0000_0000_0010_0101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b101_0000_0010_1100_0001_0010_0110_1100_1010_0000_0010_0100_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1110_0000_0000_0000_0110_1110_1000_0000_0000_0100_1010_0010_0000_0000_0000_0000_0000_0000_0000_0001_1011_1011_0111_0000_0000_0000_0011_0111_0100_0000_0000_0010_0101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1001_0011_1100_1001_0000_0001_0010_0000_0010_0101_0100_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1101_0000_0000_0000_0110_1101_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_1101_0100_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0001_1100_0010_0100_0100_1010_0000_0010_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0110_1001_0011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_0001_0000_1010_0010_1001_1011_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0010_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0000_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_1001_1100_0000_0000_0010_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1100_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0010_0100_1100_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0001_0010_0001_0010_1000_1110_1010_0001_0000_1110_0001_0010_1000_0000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_1001_0000_1001_0010_0000_1001_1010_0100_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0010_1001_0000_0000_0000_0011_0010_0000_0010_0101_1100_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_0100_1100_0010_1000_1110_1010_0000_0010_0101_1110_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0110_0100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_1000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0010_1011_1000_0000_0000_0011_0011_0110_1000_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0010_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0000_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0010_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0000_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1100_0010_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0001_1100_0010_0100_0100_1010_0000_0010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0110_1001_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_0001_0000_1011_0000_1001_1011_0000_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0010_0100_0011_0010_1011_0100_1010_0000_0010_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0110_1001_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0010_1100_1100_0000_0000_0010_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0100_1100_0010_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1001_0110_0010_0101_0000_0101_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1001_0110_0100_0101_0000_0101_1010_0000_0010_0110_1010_0011_0101_0000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_1000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0010_1101_0100_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_1001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_0001_0000_0000_0000_1001_1010_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0000_0010_0110_0100_1000_0000_0100_1010_0000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_1000_1000_0110_0100_1110_1100_1101_0010_0000_0000_0000_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0010_1101_1000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_1110_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0101_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_1001_0110_0000_0000_0000_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0101_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0000_0000_0000_0110_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_1001_0110_0000_0000_0000_1100_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0101_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0000_0000_0000_1010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0001_1001_0110_0000_0000_0001_0100_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0101_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0000_0000_0000_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0101_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1101_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1000_0000_0000_0001_0001_0110_0000_0000_0000_1000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0001_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0000_0010_0110_0100_1000_0000_0100_1010_1111_1111_1111_1110_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0111_1111_1111_1110_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1100_0100_0000_0000_0001_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0000_1100_0100_0000_0000_0010_1011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0110_1000_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1001_0000_1001_1000_0000_0000_0011_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0010_1100_0000_0010_0101_1010_0101_0000_0100_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1110_0000_0000_0000_0000_1110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1110_0000_0000_0000_0000_1110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_1001_0000_0000_1000_1000_1001_1100_0000_0000_0000_0001_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1001_0000_1110_1100_0000_0000_0010_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_1100_0000_0000_0000_1101_1011_0110_0000_0000_0001_1010_1100_0000_0000_0001_1011_1000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1011_0110_0000_0000_0001_1010_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1000_0100_0011_1000_0100_1101_1011_0000_0000_0000_1010_1000_0000_0000_0000_1010_1111_1111_1111_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_1001_1111_1111_1111_0011_0110_1100_0000_0000_0010_1001_1111_1111_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_1000_1001_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0000_0100_0000_0011_1001_0000_0100_0000_0010_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_1101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0000_0100_0000_0010_1001_0000_0100_0000_0010_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_0111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_1101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0000_0100_0000_0011_1001_0000_0100_0000_0011_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_1101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0000_0100_0000_0010_1001_0000_0100_0000_0011_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_1101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_0100_1111_1111_1110_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1010_1100_1000_1000_0010_0010_0100_1101_1010_0000_0000_0000_1101_1000_0000_0000_0000_1101_1001_0000_0000_0000_1010_0111_1111_1111_1000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0010_0010_0000_0100_1000_1000_1000_1101_0100_1111_1111_1110_1011_0100_1111_1111_1111_1001_0100_1111_1111_1111_1111_0001_0000_0010_0100_0100_0100_0110_1010_0111_1111_1111_0110_1010_0111_1111_1111_1010_1010_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_1001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_1000_1001_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0000_0100_0000_0011_1001_0000_0100_0000_0001_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_1101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0000_0100_0000_0010_1001_0000_0100_0000_0001_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_1101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0000_0100_0000_0001_1001_0000_0100_0000_0011_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_1101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0000_0100_0000_0001_1001_0000_0100_0000_0010_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_1101_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_0100_1111_1111_1110_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1010_1100_1000_1000_0010_0010_0100_1101_1010_0000_0000_0000_1101_1000_0000_0000_0000_1101_1001_0000_0000_0000_1010_0111_1111_1111_1000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0010_0010_0000_0100_1000_1000_1000_1101_0100_1111_1111_1110_1011_0100_1111_1111_1111_1001_0100_1111_1111_1111_1111_0001_0000_0010_0100_0100_0100_0110_1010_0111_1111_1111_0110_1010_0111_1111_1111_1010_1010_0000_0000_0000_0000,0b010_1001_0000_1100_1100_0000_0000_0011_0010_1011_1111_1111_1111_1110_1100_0000_0000_0000_0011_0100_0000_0000_0000_0010_1011_1111_1111_1111_1111_0100_0100_0000_0000_0011_0110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0001_0001_1000_0000_0000_0000_1010_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_0001_0011_0000_0100_0000_0010_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0111_1111_1111_1111_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1111_1011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1001_0001_0001_0100_0000_0000_0011_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0100_1000_0000_0000_0011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0110_0100_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0001_0100_1101_1011_0010_0000_0000_0001_0110_0000_0000_0000_0101_0010_1000_0001_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0110_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_1011_0000_0000_0000_0010_1011_1111_1111_1111_1000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1011_0000_0000_0000_0001_0101_1111_1111_1111_1100_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_1010_0000_0000_0000_0010_1011_1111_1111_1111_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1101_1000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_1010_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1101_1000_0000_0000_0001_0101_1111_1111_1111_0110_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_0000_0000_0000_0010_1011_1111_1111_1110_0100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1101_0000_0000_0000_0001_0101_1111_1111_1111_0010_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1110_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1101_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1110_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0111_1111_1111_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1100_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0110_0100_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0001_0100_1101_1011_0010_0000_0000_0001_0110_0000_0000_0000_0101_0010_1000_0001_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0110_0000_0000_0000_0010_1100_0000_0000_0001_0010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0001_0100_1101_1011_0000_0000_0000_0001_0110_0000_0000_0000_1001_0010_1000_0001_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0110_0100_0000_0000_0010_1100_0000_0000_0001_0010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0001_0100_1101_1011_0010_0000_0000_0001_0110_0000_0000_0000_1001_0010_1000_0001_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_1011_0000_0000_0000_0010_1011_1111_1111_1111_1000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1101_0000_0000_0000_0001_0101_1111_1111_1111_1100_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1011_1111_1111_1110_1110_1011_1111_1111_1110_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0101_1111_1111_1111_0101_0101_1111_1111_1111_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1011_1111_1111_1110_0110_1011_1111_1111_1101_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0101_1111_1111_1111_0001_0101_1111_1111_1110_1110_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_1011_0000_0000_0000_0010_1011_1111_1111_1110_0100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1101_0000_0000_0000_0001_0101_1111_1111_1111_0010_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_0111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1110_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1101_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1110_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0111_1111_1111_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1100_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1101_0101_0000_0000_0000_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0001_0100_0100_0000_0000_0000_1010_0000_0001_1110_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0001_0011_0000_0100_0000_0010_0011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0001_0100_0000_0000_0000_0001_0011_0110_0100_0000_0000_0011_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_1100_1100_0000_0000_0001_0011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0001_0111_1100_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0010_1010_1000_0110_0011_0101_0100_1101_1000_0000_0000_0000_1101_0001_0000_0000_0000_1100_0000_0000_0000_1000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_1001_0000_1100_0110_0001_1001_0100_1111_1111_1111_1111_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_0011_0100_0011_0001_1000_1011_0010_0000_0001_1111_0010_0010_1010_0000_0000_0001_1011_0110_0100_0000_0000_0011_0110_1000_0000_0000_0010_1000_0000_0000_0010_0110_1000_0000_0000_0100_0010_0010_0011_0100_1001_0110_0001_1101_0100_0000_0000_0000_0101_1011_0110_0000_0000_0001_0101_0000_0000_0001_0111_0001_0001_1010_0100_1011_0000_1110_1010_0000_0000_0000_0011_1101_1011_0000_0000_0000_1010_1000_0000_0000_1110,0b000_0000_0010_1000_0000_0100_1000_0100_1010_0000_0001_1111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0110_0100_0000_0000_0000_1001_1011_0100_0000_0000_0001_0101_0000_0000_0001_0010_0000_0000_0000_0000_0001_0010_0011_0010_0000_0000_0000_0100_1101_1010_0000_0000_0000_1010_1000_0000_0000_1100_0000_0000_0000_0000_0000,0b000_0001_0001_0111_0000_0000_0000_0000_1010_0000_0001_1111_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0110_0100_0000_0000_0010_1010_0000_0000_0000_0110_0101_0000_0001_0000_0000_0000_0000_0000_0000_0001_0001_0001_0110_1101_0010_0000_1100_1101_1011_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_0000_0000_0000_0010_1001_1111_1111_1111_1000_0000_0000_0000_0000_0001_0100_1101_1011_0010_0000_0000_0001_0101_1111_1111_1111_1111_0010_1000_0000_1000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_1001_1111_1111_1111_1110_1011_1111_1111_1111_0110_0101_0000_0001_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1011_0000_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0110_0100_0000_0000_0010_1011_1111_1111_1110_1110_0101_0000_0001_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1100_0000_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0111_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1101_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1101_1011_0010_0000_0000_0001_0101_1111_1111_1111_0011_0010_1000_0000_1000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_1001_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0100_1001_0100_1111_1111_1110_0111_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1101_1010_0000_0000_0000_1010_1111_1111_1111_0110_1011_0000_0000_0000_0000_1101_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0100_1000_1000_1000_1001_0100_1111_1111_1111_1001_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0001_1001_0100_0000_0000_0000_1010_0000_0010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_0001_0011_0000_0100_0000_0011_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0011_0110_0100_0000_0000_0011_0110_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0001_1010_1100_0000_0000_0010_1011_0110_0000_0000_0000_0010_1001_1111_1111_1111_1111_0110_0100_0000_0000_0011_0100_1000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1000_0000_0000_0011_0000_0000_0000_1010_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1000_0000_0000_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1000_0000_0000_0011_0000_0000_0000_0100_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1100_0100_0000_0000_0000_1010_0111_1111_1111_0111_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1110_0110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1100_0000_0000_0001_1010_0000_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0001_1100_0100_0000_0000_0000_1010_0000_0010_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_0100_0010_0100_1000_0010_0011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0011_0110_0100_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_1001_0011_0101_0010_0001_0000_0000_1011_0011_1000_0000_0000_0000_0011_0100_1100_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_1001_0011_0101_0010_0001_0000_0000_1001_0011_1000_0000_0000_0000_0011_0100_1100_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0011_1111_1000_1000_1000_1101_1101_0000_0000_0000_0001_0110_0000_0000_0000_0001_0010_1010_1000_0000_0001_0001_0010_0000_0000_0100_0100_0110_1110_1100_0000_0000_0000_1011_0000_0000_0000_0000_1001_0101_0100_0000_0000,0b000_1010_0100_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0010_1001_1111_1111_1111_1011_1010_0000_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0100_0000_0001_0101_0001_0100_1111_1111_1111_1111_1101_1000_0000_0000_0001_0110_0000_0000_0000_1001_0010_1000_0010_0000_0000,0b000_0010_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1010_1100_1010_0000_0000_0000_0001_1101_1000_0000_0000_0000_1101_1001_0000_0000_0000_1101_1010_0000_0000_0000_1101_1011_0000_0000_0000_1101_1100_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0010_1001_1111_1111_1111_1011_0110_1000_0000_0000_0010_0101_0000_0010_0000_0010_0101_0000_0100_0000_0001_0100_1101_0100_1111_1111_1111_1111_1011_0100_0000_0000_0001_1011_0110_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0011_1101_1000_1000_1000_1101_1101_0000_0000_0000_0001_0110_0000_0000_0000_0001_0010_1010_1000_0000_0001_0001_0001_1111_0000_0100_0100_0110_1110_1100_0000_0000_0000_1011_0000_0000_0000_0000_1001_0101_0100_0000_0000,0b000_1010_0100_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0010_1001_1111_1111_1111_1011_1010_0000_0000_0000_0010_1100_0000_0000_0000_1010_0101_0001_0000_0000_0001_0101_0001_0100_1111_1111_1111_1111_1101_1000_0000_0000_0001_0110_0000_0000_0000_0101_0010_1000_1000_0000_0000,0b000_0010_0010_1000_0000_1000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0011_1110_1000_1000_1000_1101_1101_0000_0000_0000_0001_0110_0000_0000_0000_0001_0010_1010_1000_0000_0001_0001_0001_1111_1000_0100_0100_0110_1110_1100_0000_0000_0000_1011_0000_0000_0000_0000_1001_0101_0100_0000_0000,0b000_0010_0010_1000_0000_1000_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0010_1001_1111_1111_1111_1011_1010_0000_0000_0000_0010_1100_0000_0000_0001_0010_0101_0000_0100_0000_0001_0101_0001_0100_1111_1111_1111_1111_1101_1000_0000_0000_0001_0110_0000_0000_0000_0101_0010_1000_0010_0000_0000,0b000_0010_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_1001_0000_0000_1000_1000_1001_0100_1111_1111_1111_1101_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0111_0101_1000_0000_0000_0000_0111_0100_0000_0000_0000_0100_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0100_0001_0000_1000_1000_1101_1100_1000_0000_0000_0001_1101_0000_0000_0000_0001_0110_0000_0000_0000_0101_0010_0100_0001_1000_0000_0000_0110_1101_0010_0000_0000_0000_1101_0101_0000_0000_0000_1011_0000_0000_0000_0000,0b000_1010_0100_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_1111_1010_0101_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1101_0100_1111_1111_1111_1111_1010_1000_0000_0000_0001_0010_1000_0010_0000_0000_0000_0000_0000_0000_0000,0b001_1001_0010_0100_1100_0000_0000_0010_0011_0001_0000_0000_0000_0011_0000_0000_0000_1010_0010_1001_1111_1111_1111_1111_0111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_1100_0000_0100_1000_0111_1101_1100_0000_0000_0000_0001_1011_0000_0000_0000_0001_1011_0010_0000_0000_0001_0100_1101_0100_0000_0000_0000_0001_1011_1000_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0010_0000_0000_0000_1101_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_1011_1000_0000_0000_0000_1010_0000_0010_0011_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0001_1101_0000_0000_0000_0001_1011_0110_1000_0000_0000_0011_0110_1100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1110_0000_0000_0000_0000_1110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1000_0000_1001_0001_1101_1100_0000_0000_0000_0001_1011_1000_0000_0000_0001_1011_0110_0000_0000_0001_0000_0010_0100_0000_0100_1000_1110_1110_0100_0000_0000_0000_1101_1011_0000_0000_0000_1101_1100_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_1001_1111_1111_1111_1010_1100_0000_0000_0000_1010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0001_0100_1101_0100_1111_1111_1111_1111_0110_0000_0000_0000_0101_0010_1000_0001_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1010_0000_0000_0000_0000_1010_0111_1111_1111_1110_1101_0001_0000_0000_0000_1010_0111_1111_1111_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0010_0101_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1110_0000_0000_0000_0000_1110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1000_0000_1001_0001_1101_1100_0000_0000_0000_0001_1011_1000_0000_0000_0001_1011_0110_0000_0000_0001_0000_0010_0100_0000_0100_1000_1110_1110_0100_0000_0000_0000_1101_1011_0000_0000_0000_1101_1100_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_1001_1111_1111_1111_1010_1100_0000_0000_0000_1010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0001_0100_1101_0100_1111_1111_1111_1111_0110_0000_0000_0000_0101_0010_1000_0001_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1010_0000_0000_0000_0000_1010_0111_1111_1111_1110_1101_0000_0000_0000_0000_1010_0111_1111_1111_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0010_0101_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1110_0000_0000_0000_0000_1110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1000_0000_1001_0001_1101_1100_0000_0000_0000_0001_1011_1010_0000_0000_0001_1011_1000_0000_0000_0001_0000_0010_0100_0000_0100_1000_1110_1110_0100_0000_0000_0000_1101_1100_0000_0000_0000_1101_1101_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0011_0100_1000_0000_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0001_0100_1101_0100_1111_1111_1111_1101_0110_0000_0000_0000_0101_0010_1000_0000_1000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1010_0000_0000_0000_0001_1010_0111_1111_1111_1111_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_1001_1111_1111_1111_0110_1100_0000_0000_0000_1010_0101_0000_0010_0000_0000_0000_0000_0000_0000_0001_0100_1001_0100_1111_1111_1111_1101_0010_1000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_1001_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0111_0111_0000_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1000_0110_0100_1111_1100_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0001_0110_0100_1000_0100_1010_0000_0010_0101_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0010_1000_0000_0000_0000_0010_1011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_0010_1100_0000_0100_1000_0101_0011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1000_0110_0101_0100_0100_1101_1001_0000_0000_0000_1010_0000_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_1001_0000_1001_0010_0000_1001_1011_0100_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0001_0110_0100_1000_0100_1010_0000_0010_0110_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0010_1010_0100_0000_0000_0010_1011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0010_1001_1111_1111_1111_0110_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0000_0100_1000_0100_1010_0000_0010_0110_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0001_0110_0100_1000_0100_1010_0000_0010_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0010_1010_1100_0000_0000_0010_1011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0010_1001_1111_1111_1111_1110_1100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1000_0110_0100_1111_1100_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_1001_0000_1001_0010_0000_1001_1010_0100_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0010_1010_1000_0100_0101_1100_0100_1011_0000_0000_0000_0000_1010_0000_0000_0000_0000_1011_0000_0000_0000_0010_1001_0101_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0101_0001_0010_0001_1001_0100_1111_1111_1111_1111_0000_0100_1011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    mss         = [0b001_1001_0000_0100_1000_0000_0000_0010_0010_0000_0000_0011_0000_0010_0000_1000_0000_1101_0010_0000_1000_0001_1101_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0101_1000_0000_0000_0000_0101_1000_0000_0001_0100_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0001_1011_1010_1100_0000_0000_0000_0010_1100_0000_0000_1010_0010_0101_0000_0010_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_0010_0000_0000_0000_1101_1000_1000_0000_0000_0001_1010_0000_0000_0000_0001_1010_0010_0000_0000_0001_0100_1001_0100_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0000_0010_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_1010_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_0010_0000_0000_0000_1001_1011_0100_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0000_0010_0000_0000_0000_0100_1010_1000_0000_0000_0000_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0000_1110_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0001_1100_1001_0000_1100_1101_1001_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0010_0000_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0100_1000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_0010_0010_0010_0001_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0101_0001_0010_0001_1001_1010_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0001_1010_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0001_1010_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0010_1100_0000_0000_0011_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0010_0101_0001_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1000_0100_0000_1010_1100_1011_0000_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0001_1110_0010_0010_0000_1010_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_0000_0000_0000_0110_1001_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0010_1000_0011_0000_0100_0110_1011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_0101_0001_0000_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_1000_1101_0000_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_0000_1010_0100_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_1010_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_1110_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_1000_1101_0001_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_0000_1011_1100_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0000_1000_0100_1101_0001_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0001_0000_0100_1000_0100_1010_0000_0000_0011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0011_1000_0000_0000_0000_0010_0100_1010_0000_0000_0010_0001_0000_0010_1101_0010_1100_0000_0000_1010_0001_0101_0001_1100_1000_0000_0000_0001_0010_0101_0000_0000_0001_0000_1000_0001_0110_1001_0110_0000_0000_0101_0000,0b000_0000_1000_0001_0100_0100_1000_0100_1010_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_0001_0000_0000_0000_0001_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0001_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0010_0001_0011_1000_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    ordlist     = [0b000_0000_0100_1000_0000_1001_0000_0100_1010_0000_0000_0010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0000_0100_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_0100_0001_0000_0000_0101_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0001_1000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0001_0000_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0000_1110_0100_1000_0110_0011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0010_1000_0001_0000_0001_0100_1010_0000_0000_0010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0000_1100_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1010_0000_0000_0000_1001_1010_0100_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_0010_0000_0000_0000_0000_1010_0000_0000_0010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_1000_0011_0000_0011_0100_1010_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0000_1110_0100_1000_0100_1010_0000_0000_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0000_0110_0010_0010_0000_1010_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1110_0000_0000_0000_1001_1011_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0010_0000_0111_0000_0000_0000_0100_1101_1000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0000_0100_0100_1000_0100_1010_0000_0000_0011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1110_0000_0000_0000_1001_1011_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1000_0000_1100_1001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1000_0010_0000_1000_0100_1010_0000_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_0010_0100_0010_0010_0010_0010_0011_0100_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_0010_0100_0010_0010_0010_0010_1011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0000_1110_0100_1000_0110_0011_0110_0100_0000_0000_0010_1100_0000_0000_0000_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0010_1000_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_0011_1000_0000_0000_0000_1010_0000_0000_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0010_0001_0010_0001_1001_0100_1111_1111_1111_1111_0100_0000_0000_0000_0010_0000_0000_0000_0000_0001_0110_1001_1000_1000_0000_0000_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0001_0000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0001_0000_1001_0000_1100_1010_0111_1111_1111_1100_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_1000_0000_0000_0000_1011_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0001_1000_1000_1000_1001_0100_1111_1111_1111_1001_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0000_0110_0010_0010_0000_1010_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0001_1001_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0001_0000_1001_0000_1100_1010_0111_1111_1111_1110_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0000_1000_0100_1000_0100_1010_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0000_0011_1000_0000_0000_0010_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_0110_1001_1111_1111_1111_1110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    permsort    = [0b010_1000_0001_0011_0001_0000_0100_0111_0010_0000_0000_0010_0100_0010_1000_0000_0000_0001_0110_1100_0000_0000_0000_0010_0000_1000_0001_1001_0010_1100_0000_0000_0000_0010_1100_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0110_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0011_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0000_0100_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0001_0000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0000_0100_0010_0010_0001_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0001_1001_0010_0001_1001_1010_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0000_1100_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0011_0001_0010_0001_1001_1010_0010_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_0001_0000_0000_0000_0010_1011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0010_0000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_0000_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1000_0000_0000_0000_1001_1011_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_1011_0000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0000_0110_0100_1000_0100_1010_0000_0000_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_1111_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0001_1110_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0001_1110_0100_1000_0110_0011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0010_1000_0001_0000_0011_0101_1010_0000_0000_0011_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_0000_0000_0000_0110_1100_0000_0000_0000_0100_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0001_1100_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0001_0000_0100_0000_1010_0000_0000_0010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0011_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0010_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0000_1100_0100_1000_0100_1010_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0011_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0001_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_0010_1000_0000_0000_0001_0011_0001_0000_0000_0000_0010_1100_0000_0000_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0110_1000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1001_1000_1000_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0111_1111_1111_1110_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0000_1001_0010_0001_1001_0100_1111_1111_1111_1001_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0010_1000_0000_0000_0001_1011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    queens2     = [0b001_0010_0110_1000_0100_0000_1100_1100_1011_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0011_0101_0010_1100_0000_0000_0010_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0100_1101_0000_1000_0001_1011_1001_0110_0000_0000_0001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0000_1000_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0000_1000_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_0000_1000_0000_0000_0010_1011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0001_0100_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0011_0000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0000_1100_0010_0010_0001_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0011_1001_0010_0001_1001_1010_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0001_0000_0010_0010_0000_1010_0000_0000_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0101_1000_0000_0000_0001_0110_1001_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0001_0100_0101_0000_0110_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_0000_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_1000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0000_1010_0100_1000_0100_1010_0000_0000_0011_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0001_0111_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1000_0000_0001_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0000_0000_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0001_0111_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1000_0000_0001_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0011_0000_1000_1000_1001_1010_0100_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_0100_1001_0000_1100_1101_0011_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0011_1100_0000_0000_0001_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0010_0010_0100_1000_0100_1010_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0001_0111_0010_1100_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1000_0000_0001_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0000_1100_0010_0010_0000_1010_0000_0000_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1001_1001_0010_0001_1001_1011_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_1000_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0000_0001_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0001_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0101_1000_0000_0000_0010_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0000_1110_0100_1000_0100_1010_0000_0000_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0000_1010_0100_1000_0110_0010_0001_0000_0010_1011_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_0000_1000_0010_0101_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_1000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_0000_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0000_0100_1100_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0000_1010_1100_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0001_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0001_0011_0011_1000_0000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0001_0011_0011_0000_0011_0101_1011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0010_1001_0010_0001_1001_1010_0000_0000_0000_0001_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_0010_0100_1000_0100_1010_0000_0000_0100_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_0000_1000_0001_1001_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_0000_0100_0000_1110_1001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0000_1010_0100_1000_0110_0011_0110_0000_0000_0000_0010_1001_1111_1111_1111_0110_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0010_1100_0011_0000_0101_0001_1011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0010_1001_0010_0001_1001_0100_1111_1111_1111_1101_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0001_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0010_0110_0100_1000_0101_1011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0011_0001_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    queens      = [0b001_0010_0110_1000_0100_0000_1000_1100_1011_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0010_0101_0010_1100_0000_0000_0011_0010_1100_0000_0000_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0000_1000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0000_0010_0010_0010_0001_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0001_0001_0010_0001_1001_1010_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0000_0110_0010_0010_0000_1010_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0101_1000_0000_0000_0001_0110_1001_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0001_0100_0000_0000_0010_1011_0110_0100_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0000_0100_0100_1000_0100_1010_0000_0000_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_1000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0000_0100_0100_1000_0101_0010_1000_0000_0000_0000_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0100_0000_0000_0000_0010_0100_1010_0000_0000_0010_0000_1000_0010_0111_0000_0000_0000_0000_0000_0001_0110_1001_0000_1000_0000_1000_1001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_1000_0011_0000_0010_0100_1010_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0001_1010_0101_0000_0100_1010_0000_0000_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_1000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0010_1100_0001_0000_0010_1010_0010_0000_0000_0011_0000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_1000_0000_0000_0110_1100_1000_0000_0000_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0011_0000_0001_0000_0010_1110_1010_0000_0000_0011_0100_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0100_1000_0000_0000_0011_0110_1100_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1101_1000_0000_0000_0110_1100_1000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0011_0000_0001_0000_0011_0010_1010_0000_0000_0011_1000_0011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1101_0000_0000_0000_0110_1100_0000_0000_0000_0100_1010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0011_1100_0000_0000_0010_0011_0110_0000_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0100_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1000_0000_0000_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1011_0000_0000_0000_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0000_0110_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1000_0100_0000_0010_0100_1101_0001_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1000_0010_0000_0111_0100_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0001_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1000_0100_0000_0010_0100_1101_1000_0000_0000_0000_1110_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0000_1111_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_1000_0001_1010_0101_0000_0110_1011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0010_1100_0000_0000_0000_1011_0110_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_0010_0000_0000_0000_1101_0000_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0000_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0111_0110_1100_0000_0000_0000_0111_0000_0000_0000_0000_0100_1010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0000_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0111_0110_1100_0000_0000_0000_0111_0000_0000_0000_0000_0100_1010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_0000_1000_0001_1101_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    sumpuz      = [0b000_0010_0010_1000_0000_0001_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_0000_1011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0011_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0001_0011_0010_0100_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0000_1010_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0100_1100_0010_0000_0000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_0000_1000_0000_0000_0011_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0001_0000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_0000_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0100_0001_0010_0001_1001_1010_0100_0000_0000_0001_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0000_1110_0010_0110_1011_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_0100_0100_1000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0001_0010_0101_0000_0110_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1101_0000_0000_0000_0000_1101_0010_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0001_1000_0010_0100_0101_1011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_0000_1000_0001_1011_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0011_1100_1001_0000_1100_1101_0011_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0001_1100_0101_0000_0101_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0100_0000_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_0100_1100_0000_0000_0000_1010_0000_0000_1000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0000_0010_0100_0011_0000_0100_1000_1010_0000_0000_1000_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0010_0000_0100_1000_0100_1010_0000_0000_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1000_0001_0010_0001_1001_1011_0010_0000_0000_0001_0000_0100_0000_0001_1000_0000_0000_0000_0000_0001_0110_1001_0000_1100_0000_1011_1001_0000_1000_0000_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1010_1000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0010_1010_0010_0010_0001_1011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1011_0001_0010_0001_1001_1010_0010_0000_0000_0001_1011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1001_0000_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_0000_1000_0010_1100_0100_1000_0100_1010_0000_0000_1000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0001_0011_0000_0000_0000_1101_0100_0000_0000_0000_0101_0100_0000_0000_0000_1101_0101_0000_0000_0000_1001_0111_0001_0001_1000_0010_0111_1001_0100_0000_0000_0001_1001_1011_0110_0000_0000_0001_0100_0000_0000_0001_1010,0b110_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_0100_1100_0000_0000_0000_1010_0000_0000_1001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_1001_0000_0000_1001_1010_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_0101_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b110_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_1000_0011_0000_0111_1000_1010_0000_0000_1001_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_1001_0000_0000_1001_1010_0000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0111_1100_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_0010_0100_0011_0000_1000_0100_1010_0000_0000_1001_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0111_0000_0000_0000_0010_1011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0000_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_1000_1000_0000_0000_0001_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0100_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b101_0000_1000_0010_1100_0100_1000_0100_1010_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1000_0100_0010_1011_1101_0100_0000_0000_0000_1001_0110_0000_0000_0000_0001_0010_1000_0000_1000_0001_0000_0010_0100_0010_0001_0100_1110_1010_0000_0000_0000_0111_1011_0000_0000_0000_0000_1001_0100_0000_0100_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_1011_1000_0000_0000_0010_0011_0110_0000_0000_0000_0010_1100_0000_0000_0100_1010_0101_0001_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0100_1000_0000_0010_1011_1000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_1011_0100_0000_0000_0001_0010_0000_0000_1011_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_0000_0000_0000_0101_1000_0000_0000_1010_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1011_0000_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1000_0000_0010_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1100_0000_0000_0000_0110_1100_1000_0000_0000_0100_1010_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1000_0000_0010_1101_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0001_0000_1101_0110_0010_1000_0000_0000_0000_0111_0110_0000_0000_0000_0011_0110_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0010_0001_0000_0111_1011_0010_1100_0000_0000_0000_0011_0110_0000_0000_0000_0010_0001_0000_0111_1011_0001_0101_0001_0100_0000_0000_0000_0001_0110_0000_0000_0000_0001_1011_0010_0000_0000_0001_0010_1000_0010_0000_0000,0b111_0000_0010_1100_0000_0100_1000_0101_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0110_0110_0100_1000_0000_1010_0000_0000_1011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0000_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_0010_1100_0001_0000_1101_0010_0010_1000_0000_0000_0000_0111_0110_0000_0000_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0010_0001_0000_0111_1011_0010_1100_0000_0000_0000_0011_0110_0000_0000_0000_0010_0001_0000_0111_1011_0001_0101_0001_0100_0000_0000_0000_0001_0110_0000_0000_0000_0001_1011_0100_0000_0000_0001_0010_1000_0010_0000_0000,0b111_0000_1000_0110_1100_0010_0100_0101_1011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_1100_0000_0000_0010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_1000_1000_0110_0001_1110_0100_1011_0000_0000_0000_0000_1010_0000_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0111_0001_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_1000_1000_0110_0001_1110_0100_1011_0000_0000_0000_0000_1010_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0111_0011_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_0100_1000_0000_1001_0000_0110_1010_0001_1000_0110_0101_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0111_0100_0010_0010_0000_1010_0000_0000_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1000_0010_0001_0110_0100_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0110_1100_0010_0100_0101_1011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0111_0100_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0001_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_0000_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0000_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_0111_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_0000_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0010_0111_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1000_0110_0001_1110_0100_1011_0000_0000_0000_0000_1010_0000_0000_0000_0001_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1110_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_0000_1100_0011_0111_1001_0101_0000_0000_0000_0001_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_1000_0000_1010_0100_1000_0110_1011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_1000_0001_0100_0100_1000_0001_1011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0001_1110_0100_1000_0110_0010_0000_1000_0001_1011_0011_0100_1000_0000_0000_0010_1000_0000_0000_0000_0010_1000_0000_0000_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0110_0000_1001_0001_1101_0100_1111_1111_1111_1101_0100_1111_1111_1111_1111_1010_0000_0000_0000_0001_0110_1101_0000_1100_0000_1011_1001_1000_1000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0010_1100_0100_1000_0100_1010_0000_0000_1000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0010_1001_0010_0001_1001_0101_0000_0000_0000_0101_1011_0110_0000_0000_0000_0000_0000_0000_0000_0001_0000_0010_0110_0110_0000_1101_1100_1101_1100_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_1001_0000_0000_1001_1011_0100_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0000_1011_0000_0001_0010_0000_1100_1101_1101_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0110_0001_0010_1001_1111_1111_1111_1110_1100_0000_0000_0100_1000_0000_0000_0000_0000_0001_0100_1001_0100_1111_1111_1111_0101_1011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_0100_0001_1010_1001_1011_1000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0000_1011_0000_0001_0010_0000_1100_1010_0111_1111_1111_1011_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_0001_0000_0110_0001_0010_1001_1111_1111_1111_1110_1100_0000_0000_0100_1000_0000_0000_0000_0000_0001_0110_1101_0000_1100_0001_0001_1001_0100_1111_1111_1111_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_0001_0000_0000_1001_1011_0100_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0000_0010_0110_0010_0000_1100_0100_1101_1100_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_1100_0001_1001_1001_1011_1000_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0111_0001_0001_0000_0001_0111_1001_1011_1010_0000_0000_0001_0100_0000_0000_0000_0001_1010_0010_0000_0000_0000,0b001_0010_0110_1010_0111_1111_1111_0111_1010_0111_1111_1111_1011_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_1001_1111_1111_1100_0111_0100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_1000_0000_1010_0100_1000_0110_1010_1011_1111_1111_1111_1011_0110_0000_0000_0000_0010_1001_1111_1111_1111_1110_1011_1111_1111_1111_1011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_1000_0001_0011_0000_0100_0000_0011_0011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0010_1100_0000_0000_0000_1010_0101_0000_0100_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_0001_0011_0000_0100_0000_0010_1011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_0100_1100_0000_0000_0000_1010_0000_0000_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0010_1001_0010_0001_1001_1011_1010_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0001_0000_0010_0100_0010_0001_0101_0110_1010_0000_0000_0000_0000_1011_0000_0000_0000_0000_1101_1101_0000_0000_0000,0b000_0000_1000_0000_1010_0100_1000_0100_1010_0000_0000_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_0010_1001_0010_0001_1001_0100_1111_1111_1111_1111_1011_0100_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0001_0100_1001_0000_1100_1101_1101_0000_0000_0000_1101_1001_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1000_0100_0010_1000_1101_0100_1111_1111_1111_1111_0110_0000_0000_0000_0001_1011_1010_0000_0000_0001_0001_0000_0001_0100_1001_0000_1100_1010_0000_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_1001_1111_1111_1110_0011_0110_0000_0000_0000_0010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0001_0100_1101_0100_0000_0000_0000_0001_0010_0101_0000_0000_0001_0000_0100_0011_1011_1000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1100_1001_0000_0000_1001_0100_1111_1111_1111_1111_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_1010_1001_1111_1111_1101_0111_0110_0000_0000_0000_0010_0101_0000_0001_0000_0000_0000_0000_0000_0000_0001_0100_1101_0100_0000_0000_0000_0001_0010_0101_0000_0000_0001_0000_0100_0011_1011_1000_0000_0000_0000_0000_0000,0b010_1010_1100_1101_1101_0000_0000_0000_1101_1011_0000_0000_0000_1010_0111_1111_1111_1101_1101_1101_0000_0000_0000_1101_1011_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0010_0000_0100_1100_0001_0000_0000_1001_0100_1111_1111_1111_1111_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_0000_1100_0010_0110_1001_1010_1000_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1100_0100_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1000_0010_0001_0110_0100_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_1000_1001_0100_0001_1101_1100_0000_0000_0000_0001_1011_0000_0000_0000_0001_1011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1000_0100_0001_1000_0100_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0001_0000_1001_0000_0000_0000_0010_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0010_1001_1111_1111_1111_1111_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0001_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    taut        = [0b001_0000_0100_1000_0000_1001_0000_0101_1010_0000_1000_0011_0001_0010_0000_0000_0101_0101_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_0000_1100_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1000_1000_0000_0010_0010_0110_1011_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_1000_0000_0000_0011_0100_1100_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0000_1000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0010_0000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0010_0000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0010_0000_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0010_1000_0001_0000_0001_1000_1010_0000_0000_0101_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0010_1000_0011_0000_0001_1100_1010_0000_0000_0101_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_0100_0010_0100_1000_0000_1010_0000_0000_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_1000_1101_0000_0000_0000_0000_1001_0010_1000_0000_0000_1000_0100_0001_0101_1100_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0011_0110_0010_0010_0000_1010_0000_0000_0110_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1101_0000_0000_0000_1001_1010_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0011_0110_0010_0010_0000_1010_0000_0000_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0000_1101_0000_0000_0000_1001_1010_0010_0000_0000_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0011_0100_0000_0000_0001_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0100_1100_0000_0000_0010_0011_0110_0000_0000_0000_0010_1100_0000_0000_0000_0010_0101_0000_0100_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_0110_0010_0010_0000_1010_0000_0000_0110_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0000_0000_1100_0000_0000_0000_1000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_1001_0100_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_0001_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0000_1001_0010_0001_1001_1010_0010_0000_0000_0001_0100_0000_0000_0000_0010_0000_0000_0000_0000_0001_0110_1001_0000_1000_0001_0101_1001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_1000_0100_1000_0100_1010_0000_0000_0110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1010_0010_0001_0000_0101_1011_0010_1100_0000_0000_0000_0010_1010_0000_0000_0000_1010_0100_1010_0000_0000_0001_0010_0000_0110_1000_0000_0000_0100_1101_0000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_1100_0100_1000_0100_1010_0000_0000_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0011_0011_0010_0001_0000_0000_1011_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1101_0000_0000_0000_0000_1101_0010_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0011_0100_0010_0010_0000_1010_0000_0000_0111_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1001_0000_0000_0000_0101_1000_0000_0000_0001_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1101_1000_1000_1000_1001_1010_0010_0000_0000_0001_1010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_0111_0000_1001_0000_1100_1101_0001_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0011_1100_0100_1000_0101_0011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_0010_0100_0001_0000_0111_0101_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0000_1000_0100_0010_0100_1000_0101_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_0000_1101_0010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0001_0000_1001_0010_0001_1001_1010_0010_0000_0000_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1001_0000_0111_1100_0000_0000_0010_1011_0110_1000_0000_0000_0011_0110_0000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0011_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0100_1000_0100_1000_0101_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0110_1100_0100_0000_0000_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0001_0001_0000_1001_0000_1001_0000_1100_1101_0011_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0100_0110_0101_0000_0101_1011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1101_1000_0000_0000_0000_1101_0001_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_0110_0000_0000_0000_1001_1010_0100_0000_0000_0001_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_1000_0100_1010_0010_0110_0100_1010_0000_0000_0111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0110_1100_0100_0000_0001_0000_1010_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_1000_0010_1100_0000_0011_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0100_0001_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0001_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0011_0011_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1001_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0001_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1110_0001_0010_0001_1001_0000_0000_0010_1001_1001_0000_0100_0010_1000_1000_0000_0000_0000_0000_0001_0001_0000_0111_0000_1001_0000_1100_1000_0000_0001_0100_1100_1100_0010_0000_0010_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0000_1100_0000_0000_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1000_0010_0000_1001_0100_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1000_1000_0011_0100_0010_0010_0001_0011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1001_0000_0010_0000_0000_0000_0010_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1001_0000_0010_0000_0000_0000_0010_0011_0100_0000_0000_0000_0011_0110_1000_0000_0000_0011_0110_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0010_0000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0011_0100_0000_0000_0001_1011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_0011_0100_0000_0000_0001_1011_0100_0000_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0000_1000_0011_1000_0100_1000_0100_1010_0000_0000_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0001_0000_0000_0000_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_1000_0000_0000_0001_1000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1000_0010_0000_1001_0100_1110_1000_0000_0000_0000_1010_0111_1111_1111_1110_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1110_0001_0010_0001_1101_0000_0100_0001_0010_1001_1101_0000_0000_0000_0001_0100_1111_1111_1111_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1101_1000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1011_0001_0010_0001_1001_0100_1111_1111_1111_1101_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0001_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0010_0000_1011_0001_0010_0001_1001_0100_1111_1111_1111_1111_0110_0000_0000_0000_0000_0000_0000_0000_0000_0001_0100_1001_0100_1111_1111_1111_1101_0000_0100_0010_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1010_0111_1111_1111_1111_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0100_0101_0010_1011_1111_1111_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1000_1000_0011_1000_0100_1000_0110_0010_0000_1000_0010_1111_0011_0110_0000_0000_0000_0010_1001_1111_1111_1111_1110_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0001_0000_0000_0000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0000_0010_1100_0000_0100_1000_0110_1011_0110_0100_0000_0000_0010_1100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0001_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0001_1000_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_0000_1011_0000_0000_1100_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0011_0001_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0001_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_0110_0000_0000_0000_1001_0100_1111_1111_1111_1101_1000_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0100_0001_0110_0000_0001_1100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0001_0000_1010_0111_1111_1111_1110_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0100_0010_0110_0000_0000_0000_1001_0100_1111_1111_1111_1011_1000_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0010_0001_0100_1111_1111_1111_1111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000],
    while_      = [0b000_0010_0010_1000_0000_0001_1100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_0001_0000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0100_1101_0010_0000_0000_0000_1101_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0001_0000_0000_0100_0000_0000_0011_0010_0000_0000_0111_0100_0010_0101_0000_0100_0000_0011_0100_1100_0000_0000_0011_0110_1000_0000_0000_0011_0101_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0000_1100_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0010_1100_0000_0010_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0010_1000_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_0010_1000_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1000_0000_0000_0000_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_0001_1001_0011_0100_1000_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_0010_1000_0000_0000_0010_0011_0100_0000_0000_0000_0011_0110_0100_0000_0000_0010_1000_0000_0000_0000_0011_0111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_0001_1001_0011_0100_1000_0000_0000_0011_0110_0100_0000_0000_0011_0111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0001_1100_0000_0000_0011_0010_0000_0000_0111_0110_0011_0110_1000_0000_0000_0011_0101_0000_0000_0000_0011_0110_0000_0000_0000_0011_0100_0100_0000_0000_0011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0010_0100_0000_0000_0010_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1100_0000_0000_0011_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0100_1101_0001_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0100_1101_0000_0000_0000_0000_1010_0000_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_0000_0000_0000_0001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_0011_1000_0000_0000_0000_1010_0000_0000_0111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_1000_0000_0000_0110_1001_0000_0000_0000_0100_1010_0000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1101_0000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_0100_0100_0000_0000_0000_1010_0000_0000_0111_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_1000_0000_0000_0110_1001_0000_0000_0000_0100_1010_0000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1101_0000_0000_0000_0000_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_0101_0000_0000_0000_0001_0010_0000_0000_0111_1100_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_1000_0000_0000_0110_1001_0000_0000_0000_0100_1010_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1101_0000_0000_0000_0000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_0101_1100_0000_0000_0001_0010_0000_0000_0111_1110_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0011_0011_0110_1000_1000_0000_0000_0110_1001_0000_0000_0000_0100_1010_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1101_0000_0000_0000_0000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_0110_1000_0000_0000_0000_1010_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1101_0000_0000_0000_0000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_0111_0100_0000_0000_0000_1010_0000_0000_1000_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1101_0000_0000_0000_0000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1101_0000_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0100_1101_0000_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1000_0100_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0001_0000_1000_0100_0000_0000_0000_1010_0000_0000_1000_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0100_0001_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1100_0000_1101_1001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0010_0100_1101_0010_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0001_0000_1000_0100_0000_0000_0000_1010_0000_0000_1000_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0100_0001_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1100_0001_0000_1001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_1010_1101_0000_0000_0000_0000_1001_0010_1000_0000_0000_1000_0110_0001_1100_1100_1101_0001_0000_0000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1001_1000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0001_0000_1001_1000_0000_0000_0000_1010_0000_0000_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0100_1011_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1100_0001_1100_1001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0001_0000_1000_0100_0000_0000_0000_1010_0000_0000_1000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0100_0001_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1100_0001_0011_1001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1101_0001_0000_0000_0000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_0001_0000_1000_0100_0000_0000_0000_1010_0000_0000_1000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0100_0001_0011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_0000_1100_0001_0110_1001_1010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1001_1000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0010_0001_0000_0011_0011_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0100_1101_0001_0000_0000_0000_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0110_1100_0100_0000_0000_1000_1101_0001_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_1000_1010_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0010_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1000_0100_0000_0000_0001_1011_0100_0100_0000_0000_0011_0110_1000_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_0000_1101_0011_0110_1000_0000_0000_0011_0100_0000_0000_0000_0011_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1001_0000_1011_0000_0000_0000_0000_1010_0000_0000_1000_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_0001_0000_1001_1000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0110_1100_0000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0010_0010_0000_0110_0111_0011_0110_1100_0000_0000_0011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0100_1100_0010_0000_0000_0000_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1010_0010_1000_0000_0010_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_1000_0011_0110_0000_0000_0000_0011_0110_0100_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0001_0001_1011_0010_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_1101_0000_0000_0000_0010_0011_0100_1100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_1000_1101_0000_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0110_1100_0100_0000_0000_1000_1101_0010_0000_0000_0000_1101_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b110_1000_0001_0010_0001_0000_1101_1000_1010_0000_0000_1001_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1010_0010_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_0000_0100_1000_0100_0011_0110_1101_1000_1000_0000_0001_0001_0100_0000_0000_0101_0011_0100_0000_0000_0110_1011_0110_1001_1000_0100_0000_0011_0001_0110_0000_0000_0000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b111_1001_0000_0001_0000_0000_0000_0001_1011_0100_0100_0000_0000_0011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_1010_0100_1101_0000_0000_0000_0000_1101_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1101_1000_0000_0000_0000_1101_1010_0000_0000_0000_1001_0100_0001_0000_0000_1101_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0010_1010_1110_1000_0000_0000_0000_1011_0000_0000_0000_0000_1001_0100_0001_0000_0000_1101_0000_0000_0000_0000_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b010_0010_1010_1110_1000_0000_0000_0000_1011_0000_0000_0000_0000_1001_0100_0001_0000_0000_1101_0000_0000_0000_0000_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0010_0100_0010_0100_1000_0001_1011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0000_0010_0100_0000_0010_0010_0001_1011_0100_0100_0000_0000_0011_0100_1000_0000_0000_0011_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_0000_0000_0000_0000_1101_1010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0011_1111_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_0000_0000_0000_0000_1101_1010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0011_1111_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_0000_0000_0000_0000_1101_1010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0011_1111_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_0000_0000_0000_0000_1101_1010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0011_1111_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1101_0000_0000_0000_0000_1101_1010_0000_0000_0000_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1010_0001_1000_0011_1111_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_1011_1000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_1000_0000_0000_0011_0100_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0010_0110_1100_0100_0000_0000_1000_1010_0000_0000_0000_0000_1101_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1110_0011_0001_1000_0000_0100_0011_0110_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_0001_0000_1011_1000_0000_0000_0001_1011_0100_0000_0000_0000_0011_0100_0100_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0011_0001_0110_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0010_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0001_0001_0100_1111_1111_1111_1111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_1000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0011_0001_0110_0000_0000_0000_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0011_0001_0110_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_0111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0000_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0001_0001_0100_1111_1111_1111_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0110_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0011_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0110_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0011_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0110_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0010_0001_0100_1111_1111_1111_1111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0000_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0011_0001_0110_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0010_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1010_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0001_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0001_0001_0100_1111_1111_1111_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_1000_0010_1001_1111_1111_1101_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0001_0001_0100_1111_1111_1110_0111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1011_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0110_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0001_0001_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0010_0010_1001_1111_1111_1111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0011_0001_0110_0000_0000_0001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1100_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0010_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1101_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0010_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0111_0001_1000_1100_0000_0010_0001_0100_1111_1111_1111_1001_0100_0000_0000_0000_0001_1000_0000_0000_0011_0000,0b000_0010_0010_1000_0000_0010_1101_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0010_0010_1001_1111_1111_1101_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1001_1000_0100_0000_0011_0001_0110_0000_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1110_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_0011_0000_1000_0000_0010_0010_1100_0000_0000_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0010_0001_0100_1111_1111_1111_1111_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1110_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0010_0010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0001_0001_0100_1111_1111_1111_1001_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1111_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_1000_0010_1001_1111_1111_0111_1010_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0001_0001_0100_1111_1111_1011_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0010_1111_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0000_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_0101_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0001_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0000_1101_0110_0000_0000_0100_0100_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0000_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1100_0000_0000_0010_0010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0110_0000_0000_0001_0101_0110_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1111_1111_0000_0000_0000_0010_0000_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0001_1000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1110_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1111_0101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b000_0010_0010_1000_0000_0011_0010_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1101_1011_0001_0000_0000_0000_0010_1001_1111_1111_1101_1110_1001_1111_1111_1111_1100_0000_0000_0000_0000_0001_0110_1101_1000_1000_0000_0000_0001_0100_1111_1111_1110_1101_0100_0000_0000_0000_0000_0000_0000_0000_0000_0000,0b001_1010_1000_1000_0110_0001_1100_1100_1011_0000_0000_0000_1010_1010_0000_0000_0000_0000_1000_0010_0001_1011_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0010_1001_0010_1001_1111_1111_0010_1010_1100_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000]
)
rets = dict(
    adjoxo      = 88,
    braun       = 1,
    cichelli    = 1531,
    clausify    = 0,
    countdown   = 4,
    fib         = 1597,
    knuthbendix = 4792,
    mate        = 0,
    mss         = 210,
    ordlist     = 1,
    permsort    = 6,
    queens2     = 10,
    queens      = 4,
    sumpuz      = 0,
    taut        = 1,
    while_      = 2
)

Define a helper function `packOps` to package a binary program into packets, ready for DMA transfer to Heron.

In [ ]:
mask = lambda bits, run, offset : (bits & ((1<<run)-1)) << offset

def packOp (data, addr, go):
    full =   mask(go  , 1  , 0 ) \
           | mask(addr, 10 , 1 ) \
           | mask(data, 331, 11)
    words = [ (full >> (i*64)) & ((1<<64)-1) for i in range(512//64)]
    return words

def packOps (tmpls):
    raws   = [ (t, i, 0) for (i,t) in enumerate(tmpls)]
    raws.append((0,1023,1))
    raws.append((0,1023,0))
    wordss = [packOp (d,a,g) for (d,a,g) in raws]
    words  = [w for line in wordss for w in line]
    return words

Define a little driver responsible for loading the Heron core into the FPGA, transferring program binaries to it, and reading the result.

In [ ]:
class HeronOverlay(Overlay):
    def __init__(self, bitfile, **kwargs):
        super().__init__(bitfile, **kwargs)
        self.txBuf = allocate(shape=(8196+2,), dtype=np.uint64)
        self.rxBuf = allocate(shape=(1,), dtype=np.uint32)
        self.reset()
    
    def reset(self):
        rst = self.gpio_reset.channel1
        rst[0].off() # reset is active low
        rst[0].on()
        
    def run(self, golden, code):
        self.reset()
        ops = code.copy()
        txWords = packOps(ops)
        for i in range(len(txWords)):
            self.txBuf[i] = txWords[i]

        self.dma_tx.sendchannel.transfer(self.txBuf)
        self.dma_rx.recvchannel.transfer(self.rxBuf)
        self.dma_rx.recvchannel.wait()
        ret = (self.rxBuf[0] & 0xFFFF) >> 1
        if ret == golden:
            return f'Success. Returned {ret}'
        else:
            raise RuntimeError(f'Test failed! Expected {golden} but got {ret} ({bin(self.rxBuf[0])})')

Make an instance of our driver (this actually loads our core onto the FPGA)

In [ ]:
ol = HeronOverlay("./heron_ultra96.bit")

...and run one benchmark.

In [ ]:
ol.run(rets['ordlist'],codes['ordlist'])

Stress test the design by iterating through the entire benchmark suite a number of times.

In [ ]:
for run in tqdm(range(10), desc="Iteration"):
    for bench in tqdm(codes.keys(), desc="Benchmark", leave=False):
        ol.run(rets[bench], codes[bench])